# OpenFLIM Ops

## Dependencies

The OpenFLIM ops lives in `net.imagej.slim-curve-ops`. This dependency has to be present in order to use the ops. You can either import the package from your maven local or the ImageJ central repository.

In [1]:
// uncomment to import from local repo
%classpath config resolver mvnLocal
// import from ImageJ central repo
%classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
// uncomment to import from ImageJ central repo
%classpath add mvn net.imagej slim-curve-ops 0.1.0-SNAPSHOT
// %classpath add mvn net.imagej slim-curve-ops 0.1.0-SNAPSHOT-TEST
%classpath add mvn net.imagej imagej 2.0.0-rc-71

import net.imagej.ImageJ

ij = new ImageJ()
op = ij.op()
nb = ij.notebook()

Added new repo: mvnLocal
Added new repo: scijava.public


net.imagej.notebook.DefaultNotebookService [priority = 0.0]

In [2]:
// run this if dependency messed up
// %classpath reset

null

## Utility Code

Here is some utility code that helps display the multi-layer fitted images, no attention needed.

In [3]:
import net.imglib2.type.numeric.ARGBType
import net.imglib2.type.numeric.real.FloatType
import net.imagej.display.ColorTables
import net.imglib2.converter.Converters
import net.imglib2.converter.RealLUTConverter

class FancyDisplay {
    
    public channelAxis, op, nb
    
    public FancyDisplay(ij, channelAxis=3) {
        this.channelAxis = channelAxis
        this.op = ij.op()
        this.nb = ij.notebook()
    }
    
    public tableDisp(fitted, tMin=null, tMax=null, aMax=null, zMax=null) {
        def lifetimeAxis = fitted.ltAxis
        def fittedImg = fitted.paramMap
        def sampleZ = op.transform().hyperSliceView(fittedImg, lifetimeAxis, 0)
        def sampleA = []
        def sampleT = []
        for (int comp in 0..((fittedImg.dimension(lifetimeAxis) - 1) / 2 - 1)) {
            sampleA.push(op.transform().hyperSliceView(fittedImg, lifetimeAxis, comp * 2 + 1))
            sampleT.push(op.transform().hyperSliceView(fittedImg, lifetimeAxis, comp * 2 + 2))
        }

        println("Z min = " + op.stats().min(sampleZ))
        println("Z max = " + op.stats().max(sampleZ))
        for (int i in 0..sampleA.size() - 1) {
            println("A" + (i + 1) + " min = " + op.stats().min(sampleA[i]))
            println("A" + (i + 1) + " max = " + op.stats().max(sampleA[i]))
            println("Tau" + (i + 1) + " min = " + op.stats().min(sampleT[i]))
            println("Tau" + (i + 1) + " max = " + op.stats().max(sampleT[i]))
        }
        
        def pseudocolor = op.run("slim.showPseudocolor", fitted, tMin, tMax, 0, aMax);
        
        // default values from img
        zMax = zMax == null ? op.stats().max(sampleZ) : new FloatType(zMax)
        aMax = aMax == null ? op.stats().max(sampleA[0]) : new FloatType(aMax)
        tMin = tMin == null ? op.stats().min(sampleT[0]) : new FloatType(tMin)
        tMax = tMax == null ? op.stats().max(sampleT[0]) : new FloatType(tMax)
        
        def labeled = [:]
        labeled["Z"] = nb.display(sampleZ, 0, zMax.getRealFloat())
        
        for (int i in 0..sampleA.size() - 1) {
            labeled["A"   + (i + 1)] = nb.display(sampleA[i], 0, aMax.getRealFloat())
            labeled["Tau" + (i + 1)] = nb.display(sampleT[i], tMin.getRealFloat(), tMax.getRealFloat())
            labeled["Pseudocolor" + (i + 1)] = op.transform().hyperSliceView(pseudocolor, lifetimeAxis, i)
        }
        return [labeled]
    }
}

fcd = new FancyDisplay(ij)

FancyDisplay@4c92b242

## Loading Dataset

Here we use the [scifio](https://imagej.net/SCIFIO) [bio-formats](https://imagej.net/Bio-Formats) plugin to load time-resolved transient data from `input.sdt`.

In [4]:
sdtPath = "../input.sdt"

sdt = ij.scifio().datasetIO().open(sdtPath)

[INFO] Reading SDT header


The acquired dataset is actually a 4-dimensional image as we will be shown bellow. It appears purely dark because the notebook by default displays the first layer it sees.<br>
We now use the following snippet to "chop up" the dataset for demonstration. We also display the metadata for reference.

In [5]:
import io.scif.lifesci.SDTFormat

sdtReader = new SDTFormat.Reader()
sdtReader.setContext(ij.getContext())
sdtReader.setSource(sdtPath)
sdtMetadata = sdtReader.getMetadata()

// display the axis type of each dimension
for (d = 0; d < sdt.numDimensions(); d++) {
    printf("Dim #%d: size: %3d, type: %s\n", d, sdt.dimension(d), sdt.axis(d).type())
}

timeBase = sdtMetadata.getTimeBase()
timeBins = sdtMetadata.getTimeBins()

printf("Time base: %6f, number of bins: %d\n", timeBase, timeBins)

cStart = 6
cEnd = 15
tStart = 5
tEnd = 16

table = []
for (c in (cStart..cEnd)) {
    row = table[c - cStart] = [:]
    row.put("Channel", c)
    cFixed = op.transform().hyperSliceView(sdt, 3, c)
    for (t in (tStart..tEnd)) {
        sample = op.transform().hyperSliceView(cFixed, 2, t)
        row.put(String.format("%.1f ns", t * timeBase / timeBins), sample)
    }
}
ij.notebook().display(table)

[INFO] Reading SDT header
Dim #0: size: 128, type: X
Dim #1: size: 128, type: Y
Dim #2: size:  64, type: Lifetime
Dim #3: size:  16, type: Spectra
Time base: 12.500000, number of bins: 64


Channel 1.0 ns 1.2 ns 1.4 ns 1.6 ns 1.8 ns 2.0 ns 2.1 ns 2.3 ns 2.5 ns 2.7 ns 2.9 ns 3.1 ns 6 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAGDElEQVR42u1dW24jMQzLJXvJXrKLfCzQDpKMJZF62CxgoGhSjy3LelKax+Px+EGOr6+vH/ScGsTxPLAph5a51oMYeZ/NMvZxnXPlGZMu1GPHQ9MwSgCNfkyfeDF0IIdLof6E6XZAmzGMCHi4ZNKBRIjuoV0zeuvQD5d+Z/nxGg4G+E3EaoLqQAmBoOcv39/fW4tKSZcCFaDbKjfwKCb5v5dhe9r3cDvZLqx9APY136/WCNFQ4r/bnpLpohszUZoB59ZhyQsYfJjez8SIQyWAbjaJAWSQvf/+5rTRzdpFCjnXdQahMteiSOBhtxX1jCLGkZEWXXfX4M/inNKth0u0fQ9TDFXEAJMIPyRcy3xGD6Jn5dIt8w+r8TvHCxgKvOgdCFLg5diYQY8CiCh6Z1fgyXWdv8G7oD30IErFs7OZToUh8sFLU9hv5j4LoLn67M4qBcxAMgIVCDr8cCYljODjlNtVbWRuJQHYmSuJ/NSg2VliX8zVxAZQOHdQJLAaLKFijoMDQaotlBdAZZ6oCkN9P3oOgf8/DgatMVUFdC/cGNpI+jxIlld8E5oziAGkIsr3t+chdDz8Co9mQYXOujUZdkDHsDRxHTkPnWIgdUH5JKKX1+DaTL91cjiZie5JAq/MRP94JMqVYe4+76wGS3sEMTj7Fdp1ij3CamubVHSaezOtn99Jk+k1iBU9mk0q4JW4yTYOO7p/78SwRf29+m4BbXnW8OqmGP6x1aCNhJyZnsN/CYGkoVkCZIstj2XNYjbU7WwcmNqn00bEM6hOcllv8KdLuDLfbXHo6gRWl49l2X7adDSo8k5FrO6XiQXw1gqa28S9ujkrFjo6JmBRD8ggzcqhRw46KplCgJCrOOnefnV6Bu9VxW+2O7jcHwANmXr+sDe1ajnvAH71YBX+MOFE0OWdXvV6LoW4vDQ6v5hX5VVXhnoyCysOgrQhQB6NbQFWC9jjpnmSHZ/WxQwhV+UtPqk7I/3qIc3erGJlt7GKjCkpPcwXrRa/na1HPcbSnQtcDROzznGxh2oXk1U0EQlSWQMs0Wd5pZhTQsRELDLQ49mwxw1ajZ4hgj8MqYVYwy8a9LPqPUERVIx/ZZ7VFzl6rP9XeyfbMxxOzUzsWNLQd15G9C2dFalvz9pgnUKrsmuZANZV3bz691W1xTI2IaDQqR3B363bonYyvICySGCBDoISigWl7pJTiKihN0zOE7+ofHlGkMfLOOgDQ6hU415wm0LEvLNDuQxQiTc8jUQTL89fEcO2RAGRxPMYb6veTsWLKKDVwUxAQsQ9ZLlBHWv3ImBYGiCkS1GlV02h1hWpj7Cmcb0ILRczeNEzzCJJb0p4UlVQJDUekaJuQEhGRi/rwC1I2ohhyKipsEidxUsUd7+m1+xZRC4qOGQtG7urbA48oxafliEtmBY6EjqGCnEbJZOaOFWBOSq6rodAodGFeAjwTmcixHSWocnyHlaSVcs2gEfHeIyPuwNAFX8y3EBWFvHOOLwD5gQ9MH6mqqr2gGGceoh99TpYTE8BhKCzg+iEDjrhxIpoAo02aE6lJBJ4V948zRBFFX6wso2psHBr2DIKykR8f7XwwtoBJaqCkhJMmHBuRT/8qM+8cqjVr5tnYC///M8dIrUbjAsdNOmaUVwJFsFrAyuJMTEtvILkASduzBIsDRbOioMjdT8qrYoCiWTmUKDvDUTVALD752R5NIzLwEALw9xARu+dDsCTCCN/SjF7n59kk/HLq6pu9XKbFFJ105CEWN8NdUsVM56FiFIGLy6/T48VxTI5RcysGCbRBoNYYdgHneyMbswJbOJ9JqgjE8xZ3YntxtbJJ152G5jKfj3tDUVke9MdpMiBnVDrkhrdXMsuc7RxA7u/zUM9jQdJAA2sLVP61rBT/fFN1UPfti3TQs9DmUe3fbIqANBCBzptf+BU8ezDy3DdJrwHIPCs2QmSStujkm5QL+Bkt2+XDCTMCOxCDPY6dnsJpOIA8mDOYYDs9qrHhYJPuGliFmNpWHUxh0T/IRJA4r0pA2RY5Yr+SQKU1glIAsgvFgMosXMctlG3QBJA49jxDy/+1tdDW01/AAAAAElFTkSuQmCC" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAMIElEQVR42t1dAXLkRg6bp+3T9mn7s0s5dU4UbTcJgCBb2q1yxbHHM60WmwRBkPp8Pp//RV8/fvxYfh/9DPkd+/X9Xs73rK5l+m+bvmoLzV739XvkNezvpjdy8vOGrw2/ecjC2MWfPBGIcbIe8Q0GcXs//yZ1nOD765Eboax72r1Xb67BOJ7nTrs3JTKmqlF9/f7nz5/vwRSdN19xlxWw5wCk6n6g1/ptHIgRjhxCddO60f8Jdzq13odlAlysVy33AbGOxhEVb9Ad5mxfzgufjHWda+w6zU/gMhafPRtfq1nFG8mfyr4OkG8xWHFe/FS4eCL+uJ7+Cp/SgL3eQ5MimzgZ0qITexrzEH8/i2ArLu0tocMNADuu+/Ke/SzeU2/QFFLv2EfjwZjL8btAoNP1doaUJ4FgyAOccpunPY8DrDoP0dFikLK4jgziREbi9irV92vycu8QNlTSJ8SjnCRpJopBAZFWY8OUMu1pV7gzksx4uskgF81OrnPeujPQdsq7vEHM0pC5zC5UuYDmPLjtRjDpJrKeSGdQuB5/4aWT8nXfuEHK9alElT9te0NdXgV9LKePnmbU8zVUVLGTMBmrvzZnCACdyLuPFdmWf/c27X5H3cIV4jqYPifou3vt/3iAqVTq5U0UkueZSIcLQtc+UuQEkHo7QHP2aID78rwT43TlVbDniOkVmXq1RgJcf69MGrmYExWyk2BxQlgLv7YTBLr5g84eQ4eXmRCWNmQWHCu1a2xw9g+6qmZ/Gr5okqnPU8HK76pudfe+bBsXW13samUzhg08bqtpoLqBT2gmOY3o2f0UmMq5+n2VRlX/phovM4FLRwzvotq3jSFdZEVXPb0CNO9/o4SBk70GEeMohAcf2fAnEzRs99NpiRpxoM9w+0+lh7Ms52mHQfXcNlWwA6i5R8+c9lwoI6mg+xWOKgLm14wysY6uURoynQWfqPYyXOLm0xR2gc6U56lEiyLjUllMc1HofVM31DVd4zt7Yh03Uslw1AyFMCivS1UsuUOR6wwRE+GmU0iS4A9f/p1Z61vmA2RrQzMFlfKekN+ljSGn+gW6O3tdVcHJUnMrSOzg+jsrhE+r9VdPq1qMMjawPkPqtdoIxDhXCuLIeDuN0TlHifUOSPgNNYFq/MqQKkOKVLyLa5iSo9DirIO0h2NHfX4it0eFKm6dgqIAUnN6p7qKWN8sU4eeWNUVTqLpJ08LJw5uLyEzBaaYz+gcYOFQ9FR0iiVN4Imp1Z3ziE5I2DuLZU3e62O1sic92qUK5J50De5eBKg3UMk30T53FCmvcIFbwePaA3eL2O7/VzUNBYQuFUHV04Bsphrj1FiqeLPIGNWMg0mX7/v0fWiqZelF6j6ny6+2oO9AnOuE7sgoJjtxA2ZmbhHqQW+G5NvcHVMXWJ/kJSqDl5EKXMZK7srK6M1X5h+g7j661m0amFk4uskr8mJF26oPVOiQjyHXdD0c0V5lniEzrF+/fkGYp5Im3j/jg5xABJygzNVOlh2dCjcQVQcyrd7/+3XX16+8qPr8JKKur4FNpcS4M5LO5sydIV4BEmuMapEo+u/dY2QhMRN6Xr++D4r5EOAg7G7lUVGIRbxfrmn3mq9/7FQMxOB2+

Shown above are images from channel 6 through 15, time bin 5 through 16. For the rest of the demo, we choose channel 12 and perform the fit from time bin 9 to 20.

## Hyperparameter Setup

Prior to fitting, we set up some fitting parameters specifying how the fitting is done. All the settings are described below. The commented settings are optional and are set to default values.

In [6]:
import net.imagej.slim.FitParams
// import slim.FitFunc
// import slim.NoiseType
// import slim.RestrainType

// create a new fitting parameter set
param = new FitParams()
// the dataset (3D image with coordinates (x, y, t)) we choose channel 12 in this case
param.transMap = op.transform().hyperSliceView(sdt, 3, 12);
// // the iterative fitting routine will stop when chi-squared improvement is less than param.chisq_delta
// param.chisq_delta = 0.0001f
// // the confidence interval when calculating the error axes (95% here)
// param.chisq_percent = 95
// // the routine will also stop when chi-squared < param.chisq_target
// param.chisq_target = 1
// when does the decay start and end?
// param.fitStart = 9
// param.fitEnd = 20
// // the deacy model to use, in this case y(t) = Z + A * e^(-t / TAU)
// param.fitFunc = FitFunc.GCI_MULTIEXP_TAU
// // assume the data noise follows a Poisson distribution
// param.noise = NoiseType.NOISE_GAUSSIAN_FIT
// // the standard deviation at each data point in y
// // NB: if NoiseType.NOISE_GIVEN is used, param.sig should be passed in
// param.sig = null
// // initial Z, A_i and TAU_i (i = 1, 2, ...)
// param.param = [ 0, 0, 0, ... ]
// all three parameters above will be fitted
// param.paramFree = [ true, true, true ]
// // use the default restrain type
// param.restrain = RestrainType.ECF_RESTRAIN_DEFAULT
// the time difference between two consecutive bins (ns)
param.xInc = timeBase / timeBins
// // generates the image of return code
// param.getReturnCodeMap = false
// // generates the image of parameters
// param.getParamMap = true
// // generates the image of fitted data
// param.getFittedMap = false
// // generates the image of residuals
// param.getResidualsMap = false
// // generates the image of chi-squared
// param.getChisqMap = false
// the index of the lifetime axis (from metadata)
param.ltAxis = 2

param

xInc: 0.195313, interval: [-1, -1], intensity threshold: -1.000000, instr: null, noise: NOISE_POISSON_FIT, sig: null, param: null, paramFree: null, restrain: ECF_RESTRAIN_DEFAULT, fitFunc: slim.FitFuncNative@3b7b9f89, chisq_target: 1.000000, chisq_delta: 0.000100, chisq_percent: 95

All of the fitting ops takes the same parameter, the fitting parameter (`params`) and the Lifetime axis index (`lifetimeAxis`). The rigion of interest (`roi`) is optional (see below).

In [7]:
op.help("slim.fitMLA")

Available operations:
	(FitResults out?) =
	net.imagej.slim.DefaultFitII$MLAFitII(
		FitResults out?,
		IterableInterval in,
		FitParams params)
	(FitResults out) =
	net.imagej.slim.DefaultFitRAI$MLASingleFitRAI(
		FitParams in,
		RealMask roi?,
		RandomAccessibleInterval kernel?)

## Performing Image Fitting

Once everything is set up, the fitting routine can be easily started. The op will generate an `FitResults` object with all the per-pixel results assembled into images. Specifically, `resutls.paramMap` will be the image of fitted parameters if `param.getParamMap` is set to `true` (which is by default), and `resutls.fittedMap`, `resutls.residualMap`, `resutls.chisqMap` will be those of fitted data ($\tilde{y}$), residuals ($y-\tilde{y}$) and $\chi^2$ respectively if the corresponding `getXxMap` option is turned on.<br>

This images (`xxMap`) in `results` will be of the same size as the input dataset in X and Y directions. The result attributes (fitted parameters, $\chi^2$, etc.) for that (x, y) coordinate will be layed along the Lifetime axis. E.g. `results.paramMap(x, y, 0)` will be the *Z* (constant term) for the transient at coordinate (x, y), while `results.fittedMap(x, y, 4)` will be the fitted data of the 4th time bin ($\tilde{y}_4$) of the same pixel.

Here we demonstrate the most used ops:

### Initial Parameter Estimation (RLD)

In [8]:
// spin!
rldRslt = op.run("slim.fitRLD", param)

net.imagej.slim.FitResults@6f4c1a0f

In [9]:
// showing tau from 0.15 to 0.4
nb.display(fcd.tableDisp(rldRslt, 0.15, 0.4))

Z min = -37.23078155517578
Z max = 10.958125114440918
A1 min = 0.0
A1 max = 1728.772216796875
Tau1 min = 0.0
Tau1 max = 5.780399322509766
brightness_max automatically set to 1081.4703


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAd3klEQVR42u2dx47sRBuGDznnnHPOOXPIOecoISEyIi9IC8QGBEiwY4EQIHasWIG4AXbcAJfBDfSvx/of6+065Z6eGZh2z9RIpWnb5XL5y6nK27Zt2zZpbUu3BoRGAK01AmitEUBrjQBaawTQWiOA1hoBtNYIoLVGAK01AmitEUBrjQBaawTQWiOA1hoBtNYIYJztgAMOaMhsEmC4HX/88XMTi9e3CFGNa0L777//mu895JBD+t+HHnroZI899uh+77333jsg97zzzuv+77vvvpODDjpoK0uTxU7gsMMOm3kdRM7i5lo7++yzeyQfd9xx3e/99ttvstdee0122223Hsk0ni/RcVwjlmyMt8kkxMY/NIFcOw+iTjjhhKlrcDPXZwGd6yeffHL3++ijj56ce+65/fkkpJtvvrkfiz777LPP5KijjlpRdWxSdTEOA65EUkkQngNZcOxKAGesktA4ViI4Vo6DCilVkHNCVUhQSBP+c7wJpMDiHg4C8rdcD1BvueWWKjEowtf7TMY7/PDDp66dddZZ/fPpJ8KPOOKI7r+2gseoD/pyfOaZZw5KtkYA/2/HHHNM/1vgqrPhqpKbOIYDb7311h0402PG0aCTi7nPZ9WQUiK+JvKTOHkW4zA282H8888/v0c+z19S5G+8BBCwBx54YH8Ozk+Aw2kSBMAX2Sn6uQ7Q+Z/3lkSUnkFN9WR/kCuXe16D0nbTTTdN6X2e7fOVDI0ABhoAEmhyGsadxAAneR6dKwLlLpGeauGGG25YU0wAouN+EYhtAAHA2Ryfc845U5KK/ymF6DevemsEMNBKS1+OBLg0dLJWOvpWv14kgqTkwCExPIQMEH766af3aoH50CA0n5Xqgv7MaR53UVXRCCCMpxqwPIa7rrjiik4KpISQKABoiljOgSju0ypfT9h41113nZI85Zh4IimNcpyaTeE13iNtny1DAOli+R/gajypEpI7DbLQIBqALiEoFSCQRCDqo+R8RTXna65gGXwqJQhc7rl0TfVASnf1pJNOmjrHb4hiS0sAxTuiVHEKYEE6iARIAE5DLhHBNZDIMX29HwRoQ5TPS09gKLQ8pCLwMpJza9FJnzkrVK1dYDAq4xdbigASkZdcckmHeLkHJEAcF1xwQScay0hbBmroe+GFF04uu+yybhwIIY3HWUitzWlIVWjwORZ9M+qoV5JSp5Ru/CcWsOeee+4AB4mWMU899dTe6Ny0BFCKPjgDoCUC1bMe81v9m1zI71122WWKywVmchnElufXmi7WI3EckZd2DNeMDaQrqZvKe0LgvI/H/Pd4U0qA3XfffQq5KUa1rlUL6strr712B0NR24EGl3AP13744Ycp8VsC8rTTTptJkKibmkj2eYxtviElTBJg3lPaIsYbgMEzzzzTqQF+q8a0b1CBK+UXlloCiDCBASB4cX7TUmQefPDBM/WpuhQgGuFL5Nfi/UN+umHemu1QupcrSRC9kieeeKJ7DgZf2jGM4bEShfPaMPMkmZaOAIiYmagxYKPoB8giU1FJX67V9LK6E6SpnxOhtQjhELIcq7QdyjyESJ1HZZCn8B0vuuii/r4kJOIXSjwJX6JHFYwoarh+g0/dZyQtr4N4JIJISiNpKFDDOHALgJPD06Ub8gRWQp42BOOAAAlx1hiGqSFGdT/nr7vuusmNN97Yq7oMbdMfdaMaQPxDHBiJxDq0dVQPSy8ByoCICDKSBxCOPfbYDuiPPfbYDrF7o38i6ZRTTun1J8dw2RCxgBR1u3GFWoxfPa0kUIJAnGW0ryRGagsY27gERIFkgkghAIhBAoAoQLjvrXSUcIxqjih59O+5fbp6Ig/AeKzIAxBwANevvvrq3jZIpOAqAaDrr7++DwClNICYhhI7cFrGBDhnUCaBrupZKXycWUvb008/3T8DYkAlSHAGkTjPXAxVS5j5vFley1IQgEaNHGxBBS8O59AggrSYU08aIgXR/DZBQ38QBOJMtTIu40Ec/C4DP/SBK7O8S2+kBmSepToa0se+G/dfeeWV3dx5Z6UVz4Pbcy6MJSNA6BAr9/GOmc1MOCy1BDBSZxbNc/fee+/kjDPO6IADp+OGySEA1FDvVVdd1QEHhBshFEhwXyJPoioNz5ofP9TKAFKKf43LmtUPYrUbMoKYKkRLvzQ+fWYavBqGszyh0RMACICTAAQGDgB67bXXOp2tO6SvDyFwXRcRoMkV6R1YyMm15JJ0CTNcXKabawSa/Rgfwks7gRx/IqysIeDZEl7mDUoPBoLn/egLYRvouvPOOydPPvlkNz+TTqbGDSgtHQFkBQ//L7744t66ReyCYIDw7LPP9oERAANwuRfxa4AERKsz01LXmkY9mAhKncxzV+L4WuUxiCr7lHGClAAal5kkKvt5nrnye+edd+5UBDD58ssvu9+8N3AYqn4qy9NHTQBZOaPoFkhZ9MF/JAUv/uCDD3bABNkQB8B56qmnpgI6BlHUvXAIRABxlBG5oeLStLjnWXvAPczF80QVJTQLRdLINfhjYgvp5zi6wz/99FPXn7mbE9HtRGJy36WXXtrdAxHPKlMbFQGI1IzfZ40c+l73z0JJGjbB7bff3htgcpz+vjpVC5/QMoATuJRyM7bISCSKPIC4FnGaqWsQ5flSP5tQQq1J8Lx3ejs///xz904QPDCQUfRohqTSAnMEay/jBpFOPPP1+MIaceplEAsRgMSUHohMLXijh/r13AOQGQ8gixDHpB/cw//XX3+9d79qqVt0b7k8TO4bKl7JoBEI530gXOMaSC/mxXXnToNAdHOTQCDQt956qw9zSxDMZaeddqo+f3QEoIGk8QKH8jJyeRprII5jbQNfGIQpWgEkAMUmgPtqHJyITOnDfejzMppmybcGVy3Ik6pCbtdNS+OUZ8DREt/27du754EoEQ2CiQUwvjEPfluv+Oijj3Zw4L2NGaTUwQj1nlFLADnIqJ5+vrrN9Xj047cAwgZQTQBM7ucejjMHL7KtywMwpYhMAqnl1JVMSBa421Qu9ytZsiqJeUJ8zFFOznj+m2++2Qd0mDdEiqp69dVX+7pFVxhpBPIcmIFrzJHj2267rYML99DXGMIIqolXH/xJjlZkASTFocEP+1PUAZAAssQAoFKUw7FGzAwIYQeUz3/88cf77KLzKPtdc8011bLwNDRLj6a2dkA3NyUXSGXuSAka7h1ur0QvMXPN3MiRRx7ZjWNK2OinQTJtBhlgpfWSCyOALHxIHcbvE088sXthjDUQoGjn5QAa+hDAKUprKWJyAAJxKEACEhmvrNfPXL/ZNrgMwDPfXFEE4HHTSsNL15N79NM9D0Ej0pkvyP3ll1+6iuI77rijJyhFvp4Az6EP8xBWeg1EFhnHELfEvICS8tXp/9oiCBdUgBgjg7zYO++804k7qZsXlgvgmlQBnJNoMnVcW9tHXxHKM0uOQUwzH+aBGjIJpMRyESnXGU+9nZFAOJPfRDB5luHghx9+uCNyDTkIm8a7pSrIghfu5x6eQ1yEZ3EtCYc+ps43ODA0/zJuuZIJktZMxEAEcqUGIpzEPVrHZfGmVT8GfLgPrkkRXhpxGpwZ2ctiUa47T5I2jM85c/N33XVXH7WECHkeiGUMI5i1fQQkesbCnX377be7vkT5eI5+vLGANExdf8A1kM57gnSIxXfSLhqtDZDcn2vwUp/ykvrtAB6Rb0XQSptAGCVMUW3gx2cwFoA24aShJne7SJN+SAYikIhfEzkcOxfmpiGmJU9wirGxM+hzzz339Ig09KxaSyLmHLaAno0WvoEgiMV+zmNo15J5Vj4vhAAy5MnErcETGby8lr2iWaR4LhMxaTyWkqQWdQSpIiYLS4waiiDEsUSHy6UtYKJIBH7xxRd9sSb+vNlGiMC

### Refinement (Levenberg-Marquardt Algorithm)

#### Plaint LMA fit

In [10]:
mlaRslt = op.run("slim.fitMLA", param)

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(mlaRslt, 0.13, 0.25))

Z min = -288.048095703125
Z max = 129.5683135986328
A1 min = -129.72164916992188
A1 max = 421.7679443359375
Tau1 min = -2.19531955063383654E18
Tau1 max = 1.628263575191552E15
brightness_max automatically set to 358.49054


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAF1ElEQVR42u1dzU4jMQweCsceyokTb4BEe+nbwJEH40VZzYqRZrOTxEmcxD8fkqWFLmVqf/G/nWVZlh+QawITAACQafr6+gIAQADAX3p9fYXQoQFAAADINwAsmoK3tzefALherzjJ0AAABQAAAgBAAAAIAABR6OHhAQAAgQEAgFHVBrIOgE3QocBzAKh53TCoZAmUyugRAikFFoXWLwBA4GmbdcJfXl78AiCmwr3b8vv9PhIocj746XQ6BMHj4yMJDEe/z60VOP5GZdXODgBqT3bKJrdoixLfQ2MUktEetDe5XC7qQrwcQGaYntAJFFC11G0vV5W8qeX9SY4JkdtMGGiAsZXICdU5l5moBWfvv5l6/0w7+HgAcDFDS3SQ00YjGmFWBzrjgNpOAXM7eCP8gsF8cl0KdVN/SIBMnlPHkU5OnXxuv0Ci8In2X6cGCO2at/Sx6TCQylSOcK5GgDntohA4vm17bWLJkPaw84Fy2b7asCz8PanJpMqCkZ1T3ysBpMXW59LKaw7g8/NTHgBCBlNPWM4W19p4Z+Xouap/K/eO7PI5Alj4M0e9CLpVf64A1FJ6hgYQLnhvGb2Uk0cZd498Nt0AKFXhknoQhYBNzynhcNCOuoo4hNHqcHJoho+Pjz4AuN1uoqIEznZtT42nR82mi4b4PhX3U04vHEHDiaBU5y5FeFIzmO4B0HKqS0xELC9g1Tw8Pz//+7OIXTBR8Nn/v5hWoHxv3AzotP89NYgFk7B+iWsK7cWU0tcsjaKtxZ3GSSPZPf81DRr7sm1MxVvXAEcUyRPoZQKHMDXbd6apIp2ID+sBPZ01jSBZs4NEB38uE44aPDnstQXP/b+QTWoUUOKZp0a3SuN0T3MH0wHQ2jxJjcNLFkdIbgUf5ZcwLI4Yw6DSU0x5nxpQlrSbSdmD0NnE9K9xW12+kHuWmaZr2+dwpCG+v7/HRwHULh6qzZcAhBIBjxgVr8wJyHV8OOYEe0YovX2DXoAJtr3wCYmjRTumJUqEUdPtM2qRg6Ti0i8Q2k6ldJtPVb3WGz9YxsOpwtaSkbMudGKPIA8T9yq6ZvYOC54V9wOkMnvha5oFTfVDpHxGYiq5b4u2JoFbXhbJBgDLqtrSDsEtAUQoGbfllal7fC0DZeOBUieVNwKwRrPMG8d7D4kCRtT+PZZ1Of/u+/s7DwAoK9g0ztY5HxurP8lcDKpJ8/Z23LAgQkjDAwizgWjnkgoAi4yiNHlo+9yx5/2dHJrzIBKTLtROY+nhHzRAZ+H0BO+IekkwLwCbT2H4Gqls19NIAD5lKZQ4J1Bi2rj0xPV4/n0Y3BISiwKAtmtdvBWjdkUi2H6LuwEK6gFgpHYyux+ghzqF9jIKgL3AtfUoTM6J4BRoNFmlaj/x/mCmZT/jfD770wDSu49HPFvB3mD2zBKiAM9O4GzGa1oP2/ocZpZEwembqoHthDUzbL8BU2PDNkvbHGJ2MgikbzrIPAA0hH2Cnw8nxbnPgGHO1hZ36l7DlhnMaQAIFgpNiQRiV8RJyzlIa3Rd9wMQeDSHYTU3c0gbUE09s6JIYA7Dtt63VJNlbg5R4rp4hXkB287VUSt36wpXDZdOiANAjFmxLljqzV89dvo+PT0d/jtlhmarfOL9QDI0AIVRe9PAvTF0BV2q/TonTEt3Di3S1DbntbCc6rrHFfUiAZDYKKkygUKZRywFIvWmcimUKRvrdvpS9wqUCqp02VOt3R9xyAqSTfqE37rFu2UK2GCnkY0wr/Sk7vMR+9fC73F7uCDBcy6S4rwzuKABEwCQmHUzFta1A0DqB6+9/CGnRZxvK7dZa485clYFXVNirt4RNCNXwGFnvYChWgPsW5PXa8nBHB0nWaQJaDldUrOR8AFAAACITrXtdAAACACwQLPm/Qo1DwTlfA4BTIIJIKoxa0skQM41AABtHAAQsNJMoEbS0BMQeUadDCfeiwuCD8BLwWULAIDB1Cg0AMiViUPtHBoAnrJb+gOf4iEkZMtnhQAAAABJRU5ErkJggg==" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAZc0lEQVR42t2dBcwkRROGl/9wDjhcg7u7u7u7u7u7O4e7W/DD/XCHwwKHBQlOIFjgDgkQ5s8zSW16++vpqZbZ/WCSys7Ozs7OTleXvCXdKhxbq9UqfFvV5/Zx13lyrOrcM888M+h79rG6e6/bqq5vU+rzSb3PmHFzfidlgGNuquq9+WBdgx360OsYJYYJYn839Fp1jJaTeVo5H6Lv+03NzKpz7Qfou5e6+4yRZD5mTvmvuSVHq0kxs9BCC1U+4BjuDh3QXDM+hkFCJJdGrTQ4sVpRP6h9yCk3GHNt7cxsWrT6fjdFPeW+51bqg9EaRc8991zSjPfN9hDxrmUa+xjGaQxj5rRNmjAWW6Gipemb5fqvvfZatEEVo0J6IblCmaQpW6DV1MBqH7DLC/C5YNrNnLEhDzknY3TjN1K9s5ZWjIeKfZ+bFzurQ2dKiJ0S+gzM71YxW53+74YdlSQB5Ni7774bbNTE4AB1gzfzzDMHMaU9MKHGmtb20GIZ2v/bTZugVfeDGgMpRs/ZG0aiRh2wzTPPPJUS5u2331a5ctgZPimRgvjlsqG6gQe0QsVdqG8cqsNjjbo65gWTcN1DiJcQqgo1kyYGScypGloxoj03INOkASo6OgXeDVE7GmYJHfTGbYCYP1F1bojeDfEQUiUVr/POO2+fz2eZZZb2MWydUNWVCjI1HSCq/X2fCNfqZNf+7LPPHqUzYwZbyDXTDz300GRLX+OSaqOVqYyUHbMI+RPa2aGNZKW4XvYgv/HGG87PrrnmGrWLuvPOO5fvDznkkKygmPZ55sZalN5XK2qGVt18lb4NuWFbjTzzzDNe5pt77rn7zM6JJ564GDhwYPv9kksuWWmc3nLLLcWNN95YrLbaauX7m266qdhiiy3K8+65556O7+y5556qiZLLU2gaoGvlQsi03/eJUJsmm2yy4pJLLunz/VVXXbV9zlJLLVW+MmC4gMIMDNxDDz1UDti5555bTDXVVMXmm29ebLvttsVyyy1XrLfeesU222xT/Pbbb8WVV15ZXnvKKadsX/e0005r/+brr79e7p9//vlqr0bDHDnc51RmaKVGx1KjgwcddFCfPz/mmGMW44wzTrk/wQQTVDLQfvvtV76uu+665euKK65YvPjii8XGG29cTDTRRE6mctFaa61VTDrppMU000xTvl9hhRU6Pj/jjDNKxmF/8ODB5etYY43lZIaYwQuFzUPwgloGyKWzQ25S9hHL5jWHDRtWLLHEEu1je+21VzH11FMXo402WrHyyiu3z2NAZFZ+9913xWyzzdYxENttt5168E2CGc33iy22WMf7ZZddtr2PkTvDDDOU+0svvXQtwhdq/4RmICV4Lq1gnzbVIOFB//PPP+V3mFmTTz55sf/++xebbLJJMeuss/YZmE033bQ8Z8stt2wPCszz6quvFvPPP39pBO64445Rg56LkDqxqiEEZ2gsFhA60AsvvHCUJ7D77rsXRxxxRLHPPvt0nDPJJJOU+99++63TtSMOYB4//fTT27+DbpZZmJO23377oPOnnXbaYoEFFohGU3MkjYTiMK1YTpMfCvWJF1988WLGGWfseHDLL7+86gGjp2X/vvvuK6abbrpizjnnLK976aWXFscff7zqOmLtmwRIxLVyMlCodR+CFOaSDq26H46J6/v83R122CHpoU4xxRTtfUHu8ACYrQ8++GAx33zzFV999VX5W6gMPkd9LLroosVWW23V53qHHXZYMXTo0OzSwxe51HhJOYEjr5Su

The plaint LMA fit starts with an arbitrary set of initial parameters $z=0, a_i=1, \tau_i=1$. By design, the algorithm is only able to find values that locally minimizes the residuals and is therefore harder to converge compared to the following scheme:

#### LMA fit with estimated initial values

To set the initial parameters, either use `param.param` to set an array of global initial values:

In [11]:
// Z = 0, A = 1000, Tau = 0.187
param.param = [ 0, 1000, 0.18723493814468384 ]
mlaRslt = op.run("slim.fitMLA", param)
// later fits shouldn't be affected
param.param = null

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(mlaRslt, 0.13, 0.25))

Z min = -137.22894287109375
Z max = 459.2732238769531
A1 min = -456.62646484375
A1 max = 1088.8184814453125
Tau1 min = -1.92399227597357056E17
Tau1 max = 7.7091735513491046E17
brightness_max automatically set to 921.78534


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAFiklEQVR42u2da0osMRCFR1FBEFT86xYUFyMuyrW4Ry8NDrdtu5OqPKuS78LhwjjKdOqknieZ0+l0+raMz8/Pb+uf0TlYBAgAhsb7+zsEABAAQAAAARzi9fUVAoA47u7u5iTAx8cHBMADQAoIACAAgAAAAgAIACAAgAAAAgAIYAgPDw8QAPzH09MTBAC/cXt7++e1q6urugRAfGl+aocHAFXCjOyP3NzcsJgVkkADU0uMQhkIZpSD1yEAyaRtIcxOAsqiTd4cwgCTk4zFmTj+9yXAxcUFxpq1CtAaH7KQA4CZCeDJA/T6rIkDI3bBTD2A+/t7CDAjnp+f8z3Ay8sLi0kOQNYPAegFjCQfs00ACOLUA5wNt/w/oxFbPvPZM7y9vdUhwOPjo+rB1w8fWggvxDj6nNLPL12PHmLTU60FW2P2XMG4F2y3e6zshB5G3z6voeevs/NzQsNIsd/ac15eXv5+7SAuVNn9pcNC64U+IrZj95/vAbbZfuuH7bW462e1aOBFAdREFDpClp+b4Eqes9ZaLMOdzJNG7XZkLy9RkwBWn2Wva3jQJ6Dlm/IsSzK1oKcHLHSqaLyMWGKUlN17tOtTGkW112IRhwgT/LFbrpJ4rSVDqRAQ+/0/JVsB7Iz0x2++xAyW2+rV7nZj4bB+slTKBUqz7ZTaPOYhalVCuWQocHFEvXhfmunnv7f8K7mgki6lxnt4SXh/QszYvfhY2RZ7j+Vmj+Q+hz0P8fX1NVYSuI316/Ls6LWaLVsPpFn1BPzO3492rDQmH/1uruGsi2E2t734VszseYDURFVDAMdZ/5YI9Y3cagp47s5pPEZoaimtHDxM/cwdD8+psSU1fSgJ3O76M2m2+UJqt9BgnLfVCZT2CCS1ecjgIS8U6g6u3xvrxg0gcumzu2PuVCu2CMXz82vbnb1naOm49yjvsEQCYSvZlghS0+07KgGlBJAKPaSeyqkX6KOE1XTVYsba7uQYKUJDIomItbcKStMiFoyMi/aVk7LjXJGFJOE7+ttrcsSMrm37OvEG9Uo/yQ4uMWKNZflH2b2WLN4UwSaqgBqNlVAytmdkSYyPJX7SvMGRGrjNuQDNWQFNLS/t6MVyBK0HkpSRzkJCu6ZPLIZqmjjb39vb+XtZf4iIRxq/WMnnXOjazvVLk6qQoaV1f0rbN7U0lPYMpggBOe5S0raVdvskXiPUxNHubq+6gaaHQ1NzB8nPct9bSvblKRz8nBzqXwWkGkwj+ox5jdQxs6QFPeUsQKOfT43xR4mctLuo9TyOD4CGLoroK/DQkCW0KzW5Q0qDauATUP2lX6WkXbFET6r+TUlcuSQq88aQVCWRRNGrzRNi74/lIh4MvxoStb9IIXXkK6nVJRM/KYk08wFLuoC1/EswD7B3OZLG8BJyaa6tkQyPrMV9U/cDlJ4k5qp6NaWh5FCp55NApgmglVjlSri1SWOJsAYBCoUQidCzxPRSMzdoTYCCX0Dt43y/JmnTiE8lnUXrOUAI19fXvkJAquxLG5+l00Kvdxsp7g22SwDtgEd6rjBlsDPwzab2sn9JZaCpz2NeJXbZk9XzfuYviao9UJKWciml5CBtXv9VQGkJufSY+ABCD/8EyFHgSC+DmMDAYxAgV3AhyS9GuwVEeNDH527XSrGtNHDwAA1OHA96rTs5wCgodQazCQE2FwoBR+h2PwAgBAAI4PvySQgAAVxVB8LvB4IAAAKAEAECN0oCp4iMjVmgnrp9A80mDEIIABAANBFgQgAAAQAEAKWx913Bwu6hzTJmsDjrzwMsX0vO4kwhFpmzUQLIAQAEqAsncjoMhQcAM3seFgwPACAAgAAAAgAIAIadVRx8xqHPvQE8QFlsvmwBAgzYGsUDgKlC3FwqGNCRAKh67OEfdxn6xlw/EekAAAAASUVORK5CYII=" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAA2sUlEQVR42u3dd5A1RdUG8BVzVkyYUBQwK2BOmBURMYJZkSDmnAOYE0bMCXMkmHPGhAKKOeespf7l3/PVr6ue/eadd/K9d3fBd6um7t25Mz3d5zznOed093Svra2tVWOPs53tbNWU63faaaftzp397GdvLSv/Dz2jXmb92jF1a17TrJ/f28ppPnPOs8ZcP/WeJR3zANClwDmKGSuUrrLq1/UBaWxd2to2dG/XNXOAsAzZzQLAmIZPsdAxwupTXv0z5Y4R8rIFN0Up6tl3zSYoeHEGmFvpIaW10flUK+0C3JR6LsNyV3msuC5rS/NVU8+PvW8KvY5xE3OFvUqgbCLghiswROtd1ttU4FT/3BXwNVmkrdy2Og8FeV3PGALP3N+2yLEaamy6jSFrbnMNizLHFLD2uZYp7mkI6MtyW5vmApZV8T5wTRXcIuBoPnMs2y1yJBXeIm5hYyLkMTn40G/nOMc5RqdsdUscm5ksyiZTXdyWCD6X0akxtk9gqmU2zwFAs+y6cpuW1RWDTAHdVKAsw7o3NFWcA4AxPjHKGfJ3TX+foK7N2iPc+j1jrM/19R7IevCYz6FOpz6mOZOmf//vAuYgrs16pvYGtl3TFb2Piejr917xilfcro59AWZXB06TZc6Cx7jIdQ79D/X/b1QnSlf7htxC3++LymlsPLSlGKDNEtoGSsbQ8pTBni6/2hUPNPsG8hu34qhfp+y6G+h6dl/Atqogc0NcQ1fDxwY1XXlzm29fZg9ePb5QR0ezHVFuFH/uc5+7Ote5zrVd+5oK7HITc4LbsQHepnUYjaX/KQFP06Ka9w6Boc9HR7HN8s55znOWz/Oc5zzr1+Sg9ADBdT6d9z2f7gEQnylriMkWVdoqhpU3pB+gjR3G9NvXI/uhbteUHQuNsuqUXk8Po2Sfl7jEJcr1F7zgBavzn//8RbGA4Tjvec9bDt+dd+/5zne+cm1+d96nsgKaOqDq7e9jzz5Ln2p0KwPClM6bJrWPpfc+lsi9ETKBUwTh+/8iF7lI+U4huY6CKDbKvfjFL15d6UpXqi51qUtVV7jCFao999yz2m233arLXOYy1S677FIygitf+crVrrvuWj79v8cee5TPy172stXOO+9cjktf+tLV5S53uXKfsi52sYsVcHiOegQYzoUlNspvr5AJFuuZ6krF6lbclVLV/XOE7PCdQnz36X+gcM8FLnCBogDKuehFL7r+eZOb3KQonYKvcY1rVNe85jWrW93qVtXNb37z6ha3uEX57vNOd7pTdcc73rH87/O6171uAcG1rnWt6jrXuU514xvfuLr61a9eXe1qVysAASyH7wDi+1WvetUCNHWL2xjbAbQFB4jWetOkRQZ+unrxIjDnWC/hX/7yl68ufOELl/8plQJZKYX6jeIBgfIveclLFsHf8IY3LBaO8m90oxtV97jHPco59/r+jGc8o3rQgx5UPfzhD6/uc5/7VA9+8IPL5xOe8ITqsMMOK/cAwV3vetdq7733LsDxOxDc+ta3Ludud7vbFeDc9KY3LeUCye1vf/tqn332KeDBNgHo3JlDi85dWIoLWGS4c2qDQvMUvfvuuxehol7Wdb3rXa8oYN999y3nb3azmxXL9rtPoCD4q1zlKoUhWCZaRtn3vve9q4c85CHVox71qOqhD31o9bjHPa565jOfWYBAsU984hPLwfpdc8ghhxQAHHHEEdUtb3nLAhYKv+1tb1vqcNBBB1X3u9/9yr2uc82BBx5Y3f3udy/fDzjggPIJdFhhyC3M8eernN00ygV0KbtrHH7ICigLZaNZSiZoFs3KKZzVofNYHnZwLaXf4AY3KIpgnde//vULC/DrgIA5XPPYxz62eu5zn1sd

or use `param.paramMap` to set the per-pixel initial values from a previous fit:

In [12]:
// here we use the RLD's output as our estimation
param.paramMap = rldRslt.paramMap
mlaRslt = op.run("slim.fitMLA", param)
// later fits shouldn't be affected
param.paramMap = null

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(mlaRslt, 0.13, 0.25))

Z min = -0.3008432686328888
Z max = 5.321786880493164
A1 min = 0.0
A1 max = 1088.684814453125
Tau1 min = 0.0
Tau1 max = 3.395394802093506
brightness_max automatically set to 921.2601


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAuu0lEQVR42u3d1ZIsWRXG8YM7DO4wuLs7DO7u7u7uNri7+wWugQVBBBfwDLwAwQVPUcRvB/9DTp2q7uruqpYz1REZVZ2VuWV931p77bXt1KlTp2aHeV384hdfSzoXuchFTlyZj+l1cgs/BebSl770BX67/OUvv9a85tM/jOuiF73ohYsA81p9sYtdbGOAbq9jQIDDNOPbawMEOCQTdWxIc5YSdvc2br9O0CUveckLfN8rYabPL2sCLnWpS22dvMO0AIfpDK2S19RPOCme/zEj3PHzAdah1WcDOCeuF7CuAl/iEpdYazlWsSTrKHtpHEWXcaME2M352UsX7XKXu9yxdMLWpfkn0J/YTBu902+L2u2pw7hMkKU5bSLmNbd05skyJek0/wu5A7ieJmA3Ia5qEoG5aUD2S9qzvhu4Xy//KD31vVzHOZJ4xFZoddB3K+i6NaRynXPOOTs6hzs5cNO6Kf9883HZy1521/LvVu+DAHjlK1/57OsGEvp8V+4oWK4cQAV0Vugyl7nMrtZpGRF2cmBPmOe/WQLMg73O/myATYGbApa1COhlVgNBpeGz8h4mSY+R87n//u5eKgGkwNnJ+++ammZAacMXve+6whWuMABXnqtd7WqniXCVq1xlaK3v8llUXgSQFw2WDpMsv4ij3H33/jSN/YB4DC3F8XB8/E/wU5CBVlNSWxk4BHnd6153dqUrXWkAHTgAv/a1rz3eQwq/I6y0AIwg17ve9cZznnH/mte85kjbs9654hWvOPIoH89OSeGZs6hHsN5uVdZhWZdu/t50kAggN7zhDQdQiEDQ883HVa961aGxtceIwToAR1p+B2hkAhRC3PzmNx/33ZPH9a9//dME8wxiIIBnEMp37+Q/eCbiyD+rst/A1jICHQGxzgRx2r7upwu3qBKLTJ97gLv61a8+hOk7APwPBBdgCJlW+s2nC+iIAkDffd7hDncYpLnGNa4xu+lNbzq77W1vO969z33uM7vRjW40yHGzm91skOZa17rWSB/ILIn/5S8t+QBe3v5XtiyI9BHIfe9Is27mUQyPn8gmYGraCZlAgRIRau8JmKZ6Jm3z3J3udKdBAgL3Tqb63ve+9+zcc8+d3fKWt5zd8Y53nD3kIQ8Zzz7ykY+c3fWud509/OEPH58sAg2n0a7b3/72Iy/kQAgEAqw0fb/JTW4yCIQw3pe3fD3vPmJE4LOSAIuswDLTt0j7p+8DjSDdAy6gCdo9wCEGwAHtd8AA1Xf3gJeWAoI5dgHhxje+8bACPh/xiEfM7n//+4/0zzvvvAHoLW5xi9l1rnOd2YMf/OABnufufOc7j3wB71nPyFN6iOk3n7e61a1G+ZUD0NK5wQ1uMN5BgqzGMsdwHeTYkAP5/yjZOtofFd2p2fAbkwtwnwClTT4JF1C0zgV4z/uNsAl9qqW0tzJHjNKktZ71XqAigu+0X9r3ute9xu/eiWwIIR+gu6ShrPKi/d5lMaRz97vffZAICTQ5yo6UB408HnIXcW8h1P2QhFbXXpcGwQOSgG9zm9sMDdJmE6Lr1re+9WmiED4N9AxBIwyAgcHUI4n0vee3rIL3WBP3AO4ZIBOwpgHQtBZoynC3u91tfEcG4Ptdnve4xz1G2qxLBMzZ9N5jHvOYUT7vSdt9vsJe5jUcZIxi7QSYMnBVwJexNiEgEjD8DxBapBm5y13uMswsLQI4wQOdWfU/4FwETOtovqaA5iGJ94GinN5xDyjAkE7OIlBYBs/IQ5tNsNJCJM8g133ve9/xLvIgC1Igi/wBH/mko4mRn7oogzS9g0TqGnCLzP9uDuMhOpSbGY/PR6h/TiPTRkASIDIQ0j3vec+hXf4HBoAATOAugHoe2EghbdpJ+zl+nDz36rvn5MkPeSIfcBCN9noeyPLxHXgskHwALH3kkGdtvrTkKa1ps4TAQK/76j3P+L2YhHruBuoRBYl2B3Y37d6NwYTLJNIsAgQozfE/8wpkQqR9tEk7TBtpVdqvvdUGe8c9wgUeT9//9fEDG2hI4H9A+wQ8At3udrcb6SiH/+WNhEy5/L3vU7nkne+RFfG8/5HaM+rIMslfvu4rv/cKLu1l0OeQiXBmm7+TE7Lst0UkAryK024akrklVFpO0LWrOXx55kDK2WKC3feO74D0O2293/3uN8BERsLPCZOW/B//+McPMNwrtoB88qPxCAg4abjk87CHPWzEDmpivK8cnpc/C6Es9RqkiRwuZUIS1ga5WIX8n4MMi2+si7lfz38ncwYE6QKcUGggMAiU8AjIlcPEjLsPcMJOK4FBkIjywAc+8LQ2Nq5A2Dx5loFFmJp6QlcO2uce4TP10ndJW3NEc5FSW+89sQLvKlNBJ5dmhhWSj3fyGYDvd/VzX5rK5jn5shDI5rfC0/PyWnTv0K51dTnmR9UASlPdBzbNoU1ABhzNo1EAZAX8njanPe4jgnusApCAXRtOwPJ56EMfOoBgFfLeXdLIs69tzoGr3PJCTCQAKEJGKGkpL2K5p4mq6QIaU+0TyBG7rqh0pYW0yuv/Bq7mZbdskOtYOYHzGp85mx8dU8GiZPWlgeyTEIGu7SWYZzzjGUNwLlYi7QE8jQGqgM5TnvKUAQgwpUuYwCwuL98cLv8XywdQ4VzlAnxRxmmTVI+EptbUABqwyuyTj5BT6T1p5Pw1fpGFYz2UR8CpXoOyyUNZd+odHCsCTJuH+fZrOujTZw4XU+m7bhIwOX60h8BoPoE/7WlPG/c8D2zCqA0mKJrlOY4ZQXoeeVgCALMmBEn4hEvrmOFMMq0rvKzsLEI9D894FpjyAlzDxsrDEiCZ9l26SIB4BZ+k5Z3iDgW1PMeKRVbEV6e6jMipDt73bqOUxzoUvBtLY3PsJjQCrD0EDo1I+/P2CYbZRxBCIGDfaUtNAOK4R4A+Cd1VxI5GIhQt5iMoS+Y0U9sgDpD8hgz1DtRNPspdT0H5Chopu/vFDiKQ35Xh0Y9+9LhXM+W+ciABq4Wk6uN/xFZWz0tTvjvJ9tBGBRc5c4u0fZEHW7SrcK02GAGAVQzfJ01m8mkFQUYSbXfmlFYAgPddZJCAAe3/l7/85YMQme5G4ppHkOUALtBpW/13RJS3Miize/V4EMq7QPR7zQMggaocfi98LF9lV7bSVl9Wj5Yj+9ve9rbRk9BFrcuoXoWT8w9WBXkTQaGJ5Tmza7dK25T5ykt2jwAJtBg9oQPNPQIiHAJT+QIo0qmPDECgSI+gWIji98XzaX3amZOVTyBfQHm/JsFvxfQrq/oV0GnMoGFoIPt0NegkzbqR7stHswRg5h4BlO8BD3jAqKc03XMBHpE8VxiZZWANpUdO64z973lZ3X5nBDdG4D5B0/LHPvaxwwoA3CcBMX9PeMIThiAClTCmfWYXzWtqlv+B/MQnPnG8h0iZ/AIxtLD3p7OGfSfkPHXPSDOHDfDuI4Z3lV1aSIJs5V13shlDfq+LCjx1dF87D3RgMvG+ew7Iys16IERdT/l7n3woxJHPDawA+1kPQFCZQcICMNCBwGyr4IMe9KDZk570pKGhtKSBFlrUQI/3Cch3JCAkGgY0WkTL60mwEt5DJCABgb/hu8t3INa9KshDi3nyfvMpP/elDQxlaPIJLUIcsqnb2chfw8M5p8CUBvBZD2B7BmEpBTnweRBZGk9+8pOHUiiX58lkKvvmNC5qjmsyahL26ydk/v+nxKf25IkqbNO4CJWQGrQBEmECEnAI4CLwpz/96UOo2tY8ef8X8fMuDfXJN3jWs541wKJpHKrGCLzbpJEsgPz9j4yIgRQI1/wBgKTRNRPSbai4dFgQADa8DHBp16zVo3HJR9lZm2YceQbh5acewC+ELS9WAfjue0b56lYeoSXY2zh1rGsSBsFGAAICNAD9/7j

_Note: if both initial value settings are present, the global values will be overriden by the pixel-specific values._

### Global Analysis

In [13]:
globalRslt = op.run("slim.fitGlobal", param)

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(globalRslt, 0.13, 0.25))

Z min = -16.0
Z max = 5.352066993713379
A1 min = 0.0
A1 max = 1058.4622802734375
Tau1 min = 0.18723493814468384
Tau1 max = 0.18723493814468384
brightness_max automatically set to 904.07117


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAA3FUlEQVR42u3dd7AlVdUF8CdiQkURUREUAwpixCwqZgyIoiAGFHMWE+asKGYMYEABcwAFs2ACFYwIZlEBs4Kx1Cr9t7/6nao1X09PxxvevMF5Vbfuux1On957rbX3CX16ZWVlpRr7uchFLlJNOX6LLbbYYNtFL3rR1rLye+ga9TLrx46pW7M+zbLyGbrmLNcaY9up9l3QZzoA2ira5YypTuozStf59ePG1nEK0Dlz6NwuB85yz7OeOzcA2ljZxdR5kN90WJfzUpbf+b+t/Kl1nPqZ4pQhsGwEB8+vALNWuu7AtjLb5HwqS7sAN0WCF8HcZX6WXJeVmWV5LPOmbm8DRds1xzquL75PicXLBMpGBNzwzY2VtSbrmg6cGp+7Er6mirSV2xUq+q4zK3hm3bdGPsuXxroRh5KlemgYqxCzOGTo3DqAxtanSZau47vC05psBYyVzUVI3qyGmwccTecOXWMRTkpTeI2EhdXJkLsM37a/a9+WW245qgXQVJqxLZN51WRqiFsTyeeY9utUKe6L3WP7CNrqBQDNc+rXajKrmcP01aWrk2hqoqsO86rmqjYVZ6nsmCZbnNPHvrZ4H2e1sT0Orp8zhn2Oj2PyyXVSx6FOpzEA3cSaf+0KMEtvWRtbxjTXmsc0Gd0HuDYHdClAHVhdzO5r6UxJAjfBz8pgE21edWgyd1nsGNOMGwJRG2hmGbOYwua+fGjVADBLRYcGVGYxVlv+UB9A6qprZL6tKRbmCys+9W5l5/SFgHromNobuYjOsKWHhq4bbxp/SidQW9/9mE6dKX3r9fxCHevJV+4pzr7YxS5W/r/EJS5R/m+2LJoOHBqxnGf8YVEDZUsBwFBSNzZhbDKqeW7XGECfcerJWjPTtv3iF794+ebkOtsda59vzvd/AOHb9rQucq7tbR06ix502mQGg/qaPEOOrO+vZ/Zdslt3cp2hcWZb5m5fAMB52223Xfm+7GUvW13qUpcq+y55yUtWW2211Xof25x76UtfuhzrNxD4OC+KEcA0laYJxDFAmRIe58nNFtIMbLuRtibYLKN6cWLOjYF9cw7j+73NNtusc4gyOcNvzoqTrnjFK1bXvOY1q6te9arV1a52tfL/zjvvXP6/ylWuUl3rWteqrnGNa1RXv/rVq2tf+9rVTjvtVPbb5uP8K13pStWVr3zlavvtty/fznNt4LjMZS6zrh6pS1Ritdi+RLWYr2eqqxOonpR1ASjMZsytt966GNk3AOR/3xzA4c7zPwfYXv/c9ra3LY7dZZddqt1226183+52t6vucpe7VHvssUd1hzvcodprr72q+973vtW97nWv6o53vGP53n333Qtwrne961XXv/71q5vd7Gblf6ABIB9gASrKkv932GGH6nKXu1xRl7pizevMjRAWpg23jhm2bUvc6r14MZhtnHed61ynGNz/2267bTH0da973bI9zMZEjveNoQDBWRyBwZx897vfvbrlLW9Z3fCGN6zuc5/7VM997nOrxz72sdUTn/jEar/99qse9ahHVfvvv3/1hCc8oXrqU59a7bnnntU+++xT3fOe9yygucENblDd//73L2UAyI1udKPy7bjb3/72BSC77rrrun03vvGNS73VKfnDLDOHpjh94QAZG4sWNWsoMZszORKjGB7TGPMmN7lJcSb2hpFkmjRH2m91q1sVgFzhClcorE9Z2P2gBz2oesxjHlM96UlPqp7znOdUL3/5y6sXvehFBQDPfOYzq2c961nVne50p+rggw+uHvzgB1d3u9vdqoc//OGlzAc+8IFFKe585ztXt771rQtYHvCAB1T77rtvUY+73vWuRTXud7/7lTKAx/GOBYyhsDBLPF/m7KZRIaDL2XW0j500aTtncRzncjLpxnBMxijGjFxjnm03velNyzdm2o6B/leW8y5/+csX5XDc05/+9OoFL3hB9eY3v7l62cteVh144IHVQx7ykHLe85///LL/EY94RHH+QQcdVFThoQ99aKmLDxDc4ha3KA4GPuoATM5RN9cAJkphn+973/veBQwArB59rZyh5vAipttNDgFj+uz7Jki09Z41w0qaaAwkPnMymWZUrGdwDsE4/3MKQ/vtfwamFNhou/9Jv+xdHFYm6SfVjuFs0n/AAQdUz3jGM8q1bBMmxHgA4rx73OMeBYwURhhwLdsowt577109+tGPrp7ylKcUIGE/lVE+FbDvYQ97WFEK9QMcSuYem2MZXfMdx3SGrcqUsL65dUPz7rqkrZ4QipOSO5n4bW5zm+IQ7A/zAAKTyL9vUo6NAIK1DIyRwoDEzjmchJEy+4QHzpQ8yhE4WrlJDDkooFIW5RFCOE156oTp4rt6AIAc4qUvfWn5AJTcgcNdGyDkEz6URIgAQEpAleqJ4dBk1tVM/hrX6u7V6xqc6ercaevsYQRSrVnF0DvuuGMxUOScs7GOoR//+McX2c8+TGNcLHvyk59cjuO4m9/85iVuO95v4QHrOFniyDlyAtuoAtaL0Y4FAufL/gERSIDH9bDaNV1PHgEocga5g5zhkEMOqV71qleVfcDwtKc9rdRLXQALaOQm7tW1hToK1dVCWCOtgZXB6VpjpnV3JYduHiMZBpsjn4xLTsVzxgvjbec06gAMfjueBAOD8znadgzlILFYGNEs04KQF1CA9CNQCB/HONcxnETqfVwLc5WJya6B3Y973OPKNe2zXQihAIAnLDheGJBIAi31AUAhKXkLsKtLV7N4DTQFx03a7OvhaptMgfXa7DJjRkiM5jRZttjMsc9+9rOLUV/ykpcUo5Ja2TtjYzznBhiHHXZY2Uaiky/4kG2Mpi6unV4/3z5Y7/pASOqdS8aFEsDCYiDE5OQYtqubbUBCIbQonve85xVgcLx6yVc42r1prWA8VRFOqA5ARI26xhjm7XtZ6GhgPeb3zaAdeszKzQIAY0i2GInBEmd9SG3knfMxSxzlVE03Mms7Y5NghlcWyQYe5wZIHKs3D+DS2uB0eQF2kmPsp0JCAoeqj2sClZaCloF8wzUBxD6S7/quDRAUR7hyPLAAI8fLJwBQ+UKMfcpzHeBM38WyptDPkQeMm7rVFe+bx2UQhjHIL2ZwqmQMk7CPvPpgl/jKmOIr5ssJqAQl4AxA4Agg0bwjyZjlOOXa5jhOIuWcrbXh2owu+ZSQ+S0BdR4FohAAgsHCkDq4btTANweK95xJoYBAPQGT7Msp0vQTfjg64cAxjk++4P6pAYUYO79wVZLEobn8YyYv1Nu4mMcgie+Yj3ESJrKO+RhOYjmckQDDN+OR63TXPvKRj6xe//rXV8cdd1w594UvfGFxGEdjuwyeYZVFDbQU0oOI7RxOcWxneHGfc4Ezddf8cwymuh5nA4L/KYv6+v9tb3tb9ZrXvKYoEXXAdKACdjlHWgyOTYgBTn0I7kU+4/7UKd3HfeMrq9hRNO7By7YEptlKYGRSSBYZkuEZhHM5kISGGe9///urE044oXTXknWOlaSJyxzMwOLtG9/4xurQQw8tbXnyCyBJJgGAs/3GPmGAcRlZGUCgDphsv0TTNqogJGSsIaqjbq7Jaa985SuL2lCihCjsVz+KkkErKuJcTHeu44UN57hPZfmmYsDnfKDrmm20UWYFj3kypy3m12fUiL0cQRYZlCQzONYwDLZjRrpojzrqqJL8YbBOHUbEFGVgMfWw/33ve19hPqMCi+MxiUOpC7XgbHGX9AaE6gEcQMLhypPNYz8lcK5mqbisvmnKcaSQkkRPXdXZftcFGGXI+NUFs92XukkSARXggIfT2cD5ygJm1xAW1XOWp6VWZT7AmNiTnq7EfajW9HHDSaQwkSxivm0+MQ6mMFpG5CiAplakXIL22te+tpzPgNjPuPb5OI9sU4/0/rm+0MD

### Multiple component fit

For multi-exponential models ($I=\sum_{i=1}^n a_i e^{-\frac{t}{\tau_i}}$), set `param.nComp` to the number of exponential components:

In [14]:
param.nComp = 2
// again, we use the RLD's output as our estimation
rldRslt = op.run("slim.fitRLD", param)
param.paramMap = rldRslt.paramMap
mlaRslt = op.run("slim.fitMLA", param)
// later fits shouldn't be affected
param.paramMap = null
param.nComp = 1

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(mlaRslt, 0.13, 0.25))

Z min = -2.5444112E7
Z max = 4695018.0
A1 min = -15.217549324035645
A1 max = 1086.9586181640625
Tau1 min = -3.8798919808122356E19
Tau1 max = 3.7239729106464054E27
A2 min = -4695017.0
A2 max = 2.5444112E7
Tau2 min = -6.610360287203018E37
Tau2 max = 5.267875201440915E37
brightness_max automatically set to 885.0121


Z A1 Tau1 Pseudocolor1 A2 Tau2 Pseudocolor2 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAz9ElEQVR42u3dY5A0XbbF8Xds27Zt27Zt27Zt27Zt256YiZmJmfk0H/PG78T9P7femuruavfz3u6IjKrOSpyz19pr74M8ecghhxwyLdoOd7jDLdx/+MMfflrpnGW29Z6/7PFHOMIR1iz/EY94xBWPWam+G922+nrbuG0/iFt5j5UM2/7tIu5GAT0IiLC+CqxlyGUq7JiO63qrXdexa/2+UwY7iDx78wRYj9csOnZellcz5lrHrpeQRzrSkXZUsQ6TBNhqL1hN1jcK0jLnrbe8h0EvX50AjLhapRf9tsjw88cd+chHXtqwy95jN0Bc61oHOWHWV4nZ+L3aubP7ZN9JvO/z15gHuv+3w7Drqed8ONmMUu1hwmze6w6mGDtv/Pmm4XaAtmxivEtE2fjJvDrPXqQMq1VmtXb7SsZYBNb8sUc5ylGWThTXKvNGQTjIQsL2e9l62/RrKc1abf61gF7L21Yi2zJqtxk13CUlXfsgXrUd2TsVmD8/IBYBsEyfwaLt6Ec/+n9dZ6PXOsxtK4G4HsNspeQtapEc73jHG/tmw8ZqXb2LpL1z1xsO/h80CXe3AFs5NrDeZuZaxy2bIB7kRNm+9vB8d+8y580em1cuo1CzTbWVykkF5mP9Sn0gq11nmWbwYYYAy3bRrgTkooRtWSKtBcIyCZmtfohjH/vYB86ZDRcr9UNsJDFdDyHWozDbmgNstaSvFUvnk7J5sBeRZrbJuYhsxzzmMcc+ShConaNHMiLYjnGMY4z9dUqlHvb5Pt+DuZuevgP33JnCzTetIsq85M97JkD6nJfwjgu4QD7xiU88/gc0stnn/6Me9ajT0Y52tLFRg8rhGKAHvk/H2N/1XWu29bAepVi2ibnnc4D5ZtuyMX5RczLD5m2zHUqMndce97jHHd+B1znHOc5xBjhAYswTnOAE0+lPf/rpJCc5yXSqU51qOtOZzjSd9rSnnU5xilOMfb6f4QxnmE596lNPpzvd6abTnOY00xnPeMbx3THuoaVx0pOedPx/8pOffJznusqsPI6p/O47O9p4kI86bo9kLYqtszIP1CQ6WWZknglcgPj/+Mc//gHvdZ7jAGG/Y3we61jHmi5xiUsMYBHhrGc963TOc55zuuxlLztd+tKXni5zmcuM75e61KWma17zmtNVrnKVsf9qV7vadIELXGCQxvHnPve5p4td7GLT2c9+9unMZz7zgeshj42y+P8sZznLOEf59pg3b4wAG+2rXk9Tbd5QSfoJT3jC4Y0MClxeB1gg5qWnPOUpD3i8zxOd6ESDQBe96EUHKP73/frXv/50kYtcZIB5netcZ3r4wx8+3fa2t53ufve7Tze96U3H9xvd6EbTAx7wgOkOd7jDdPGLX3y66lWvOo4F/jnOcY7p5je/+bjWhS984bHvile84iDQhS50oelc5zrXdJ7znGe6whWuMJ3//Ocf35UxUi5jo/Uk1Ts6HLyVcX+l3r1Z4Hk08BiQ4YFMqi94wQsOAHgog+fZpBkZeOD5zne+4YU80G8k27WAB9h73ete013vetfpfve73/TQhz50esQjHjF+e9CDHjQ98IEPnC53uctN9773vafb3e5205WvfOVxDoW4zW1uM13pSlcaAFMGZHHejW9843GcY5AF0ZAiciAdwu5BcJcPAcuQYD6JW7R/rR41XszDySyQ82IE4LnA5X2AINmk17EAsZ9sA4hH8jrnyQkoCa+8z33uMz3+8Y+fnva0p01PfvKTh+fbLn/5yx8gwN3udrfpTne603TDG95wkADIfnfdm9zkJuNevivPNa5xjeke97jHdJe73GW65CUvOcqHGIgQea5+9asPkiCzcmxXRr9NLYLVPXa9ic1qHTc2BuK9wGVcoDEqMvDam93sZsOzbnnLWw4jA88+Rie5CHC9611veJxwISRQFUShJK7j2Ec+8pHTfe9733EuBXCvhz3sYePaCIVAjpMPkPqTnexkA/Bb3epWA1DKc4Mb3GAQRQhBjOte97oDdIShAojkeP8rF4JKQJFypSbrHuw0Wnu27UYLXeJW8uZ/sR74jAkMnsfYgLj2ta894rLfeBbwhARey1v9JuFzbN7od+ALH8KB5E2fAHVANIAB/GxnO9s4FnDugwCAAjwiKcO1rnWt6Ra3uMUgGml3HO+nJM94xjOmO97xjiO8IA7ZRxSqgrj3vOc9R7mcpwzC03pGIPdUK2Cttu4y+2YHX0g+Q2uG9QlMBueZwCbdZBY5AEWCEYHRyTQD80jACRs80nm897znPe9oDQDZ5tqacdTGb0hGMXioDXnkF+UZiICMykN1lIPX8+4nPvGJg4CAF1okllTh9re//diHIP7XulAuCkCV3Bux6nQ6TLUC1tMX7jdey0MZnoEBCWjeCQRg8nDS7hNo9tsciySIABi/A5MnAw04PJUauA+jM/5s929NN2TxaZ+8Q1lSA+AhHFnn2fYjHiKQd/d/zGMeM5JKoUWLgvQrO89HLLmDRFVIUgf/Uyd5z0YHpHZAKTZ/kVmGpwQqDRDGIIlAYxReRiYBCUSxWrNMVs6bnvCEJ0z3v//9x35yrJkmwfKdBwKiWC1E+I0EA1FLwf2FmkIOIpBrXi5JUw7AAhiwQJT9Kw9QI6Pvcgf3Kzwp26Me9aihRjb3RlLEojxILvzIRRwvlCAA1aNQB4UCbGSG8CKFqPuU9+W1EjpG51UMDwjJHs8mqTZNLSR5yEMeciA0OC4Z5rVCCAMHnOPJPq+uDAgo9pNkIYUqKAMVQirqg1Su79O9bn3rW4+YjoC+I+XjHve4cQyyAv3BD37wIAfyUSbkQixNUfmNXMI+9UMerRmdS/oq9min0SFbkvx1rOQL+JjvO28QzxkeaADnUYjAi7XVed2d73zn4fUAdixjk1z7EIPsUgBgkHHXIrs8EmDIwPCI4L6SMF27vmsm2hCHvMsJEARwAKJIABfHkQy4VKl+BeArC4WyH2H1JVAdko9c8hz/I7DyP/rRjx5lRSJlRzoqQRX3VBK43kLM94HPez7ZlQTxTIYVmxkdSI997GOHgcVaRgY+7/Y7kMV0zUKg8C6gyr5f+MIXDgV4ylOecqBXjrcDlAQjkusgD1CRT3Lnd+DZSDNCIJtwUHmFDeUBPhL6rjyIKttXLnnBC17wgtEaoEzuh9RA59XkXa6CGI7l+VTlmc985qgj8IU/11Wm2dbRHnj+YeueBWDk+tUZkpcyiBjNU2syMdQ73vGO6Z3vfOcwqON4JQ/xybPJNu8Xd8mwc7XjKYrkisGBB2wJnHhLAeocsk/uQW0cK04jpZDE+yWLkkGbsjqOdysnj33uc587SOX+1AgBlQGY1KRcQ12RkjoguFbD8573vJE/CBeuhfTqjBTqKlwsM8N5Tz0buKizaHaMnjIwOglmcJ5DVnkwICkAD0MIQD7nOc850C3LO5zDC3k3UEkpo73tbW8bTa+nP/3pBzpy/IYEQK9N7jwxHwklXZQEAXgcUojN1AIBKAFlQlZkcF9xXS6hfHIUZQUqggLe7zy4sQjXB7xrqivgKZRQ4XzAC3NI5JpI9MpXvnKQDYmRaLdDwP/ef2tmAWE1o5BHnsB44jDgAVc3rESQR/i/2AgMksrotamFCOdRDJLKm8gpD9JcBJZzfEc6Mo8Q4jFQJXyAAjZSIBhiVT+DSYgB0IAv1/DdvZQBmO5BmRDGNZAOsI5xX03DV73qVcPzKYhwQTmQAtHZQv4iL2hgSu6w0sSTHVaDjT+nR8aajSP2Ag5AAMN+XgO4l73sZcNAEiKGII9PetKThncBnEcjgviMGGK5PODtb3/7kOJnP/vZ0wc+8IHhRfoPAO/6wgSQG7UTi3UAIUDgNuDEUwGovH4n3xRCrF

In [15]:
// set # of exponential components
param.nComp = 3
globalRslt = op.run("slim.fitGlobal", param)
param.nComp = 1

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(globalRslt, 10, 10))

Z min = -16.0
Z max = 3.0639078617095947
A1 min = 0.0
A1 max = 243.12957763671875
Tau1 min = 1.1072466373443604
Tau1 max = 1.1072466373443604
A2 min = 0.0
A2 max = 990.9171752929688
Tau2 min = 0.1581559181213379
Tau2 max = 0.1581559181213379
A3 min = 0.0
A3 max = 958.3338623046875
Tau3 min = 0.15660516917705536
Tau3 max = 0.15660516917705536
brightness_max automatically set to 750.06635


Z A1 Tau1 Pseudocolor1 A2 Tau2 Pseudocolor2 A3 Tau3 Pseudocolor3 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAArVklEQVR42u3dRYx1WdXG8cY+3N3d3d3d3d3d3V2DQ3ANwRmQICEQIEhCkDCDhDETEqZMGJD68jvpf/V+T59bdavqVr3V3W8lJ3XvuftsWetZstdee5/TTjvttK3jdv3f//3f1nHsl+vc5z73rmXOda5zHdv+L1zrF97PwPbzzHnOc561iH3e85535W/nP//51y57Dr8OhvbjBp6x3wHpMCX2bACss56K3fQ11xb71VhnWwBc8IIXPDQJOIvZy3OWCdjkdVBGr5Ky3RzG8TnaZRMa5iwu8esBYF3JPtnE2C9DT4apOeUDLGiEzMuoIc53vvOdqfyFL3zhA7W3BOjjPN08xzqBZ8drNy15kv2gvXnCS53dyUmcX3Pp3k0Vr0ucU87kPq/dGBDDLnCBC2wM9Re60IX2xcD6slRW/+Ztjn2+yEUust2vwH6YoFlV95J5O8nxh8OXpJPtcJ1dtcOGHPC9E+usSNDDmq1sEtyb0g57BsB8EMfVadmP2tsEUQ9zqnvJS17yZANl8ypU+f3aqP2GZWvvZNj1c8Q0MC0xMmg/kjE6gIdB0Lmzul/pPewQ90lS9wePA0TQTanFVYTYa5CmevZqk0dGG9N8XL5vklkHXXg68mngUSZ56MuoIVYRqzIYc9GLXnQqNz63BE4SWX2eEVn0f0l7zO/vVbpX/X7clo5P7+fxskmYioGrQr/uj8y+2MUutivhOVqXv/zld2wX07V7iUtcYptR1772tVcGvPq8lyDYJkzakZiAUVo3pSHWtfcFbEhrziRQuC51qUudiWmj+r/lLW+53RaGR7zrXOc624BaWhG8+MUvfgKQxuDXqE0ud7nLTf+BBAj1zed1x3zYYNnXNdq3ueoMCA3kIHZ/t8jYKvWYtPf8qP77H2FvetObbl360pfeusENbnBC2RisDqBRxv0rX/nKE+Auc5nLTL9f7WpXm56tL/5Hg8au7Dima17zmtu02q9pWAL1YWrYjUwD18nby+bOyyzZ9pGwIyDq8E1ucpNFZ2wEgv+Xvexlp75h5B3veMfpPuaMzuu97nWvretd73rT98BAupUNNEk2UM0FYkktp0FW+S3r2P9Vzx7yquXB1fwcIDtpiiUm9vzVr371E+z/WPb617/+xNwlczG3mdS/Oo1Lmate9arTb1e5ylVOILL6MM59zwGB565xjWtMkniFK1xh0gqVv81tbjP9J/VXvOIVt254wxtO5WL+6Jsctu3e6FrAJqcmS3ZOZ+cdpn4RPqJhFKImhbe4xS22tQKm+M6Zu9GNbnSCo4ZJOxFEf650pSud6b62mYZRglPn6gIGffI/UwAYc+Dd6la3mvqlz/koAHzYNN3glPRwnYwYjIgjIxAu9UYNY+pItFSyK8YoMzqCqfecM9JKStUNGJiaTfUsqST1lV+1Mul3ANXfa13rWpNmwtR8iwDjt5vf/OZT2SVVniabg/OYxQIO5kSsMh9LfsM8C2gk+EgUkomBqW4XYmMgwo8gII1s+jhtJJHa8luxdkyc94epGKeHwFTf8g0yO37DSBooAOmPdldlN+nrXgJnS2A8FgA46Nq2gSlLjce8bCSpTUqT9AiGoHnqOWNJIeDpF7OByZkLzwSAJY/a/bve9a7bbXcf2DJDGOc70F33utedfvffPUAVHwCEOWNz1vI9Ao+x7iWauKQhRnO14cWqo0NbElKAJUatKjtqF0BIiiNQU76IC1wBjLQCALsdYxAxpxDR1JeWUc530s1c3OEOd5gA4vOoPfgOY7+A2zOZLJqLVtHHMbZwHFckT6/zcD3UseOjP7D03M1udrOt+9znPtvfBXBGE2AmkCRUr+nhWF9zfgCqbOo8Zw7waBuf0xwYSJ1rp//z/j3gAQ844XsaBKhcAbN21XO7291u6qOxr3LmdgomrTK3h5YRtKlFDoONuBhJUkl7juAIjAhDulYRBhMf/vCHn+AXzNW8uX1EIonaorpHhrinDn6De5kQnzl0xpF5yjHtu7I9H0OAyHP67jMg0CbGpL3RzCzNhoyh9vcyHV9HI6w5Fd0ZXd1PVe/mwY4JDpjm8iwJoEr7TCp8T02Oc38d1wdt1x7/YfTC51KzTgRttMMYFRFvfOMbb5sB7bHd3TPXT62PAR+g0uf6zayRdmMSkkaHRz7ykVO5GBHQ6+tuUnzQ5fI1TdBm1qjrrMHd+c533nbq2G7EQFTSY9AkSHkMRVxEAoocsIjlPgYAAulHSEDw+61vfesT1HDP5a0vLSYF5ubxaYZRNcfoUXppEb6Ffgdw/Wo24B6gA8+d7nSnbbDTfEUcH/rQh55ppjL2KTAElsPOFxhmP3uzO6tQGwBIQoPGYFJlXo4YAJCj1XOI25SNyRhnAkll9efgAVV2vbbmC0NjhtCotYxHO5hEA/ScfgNCfQOoUYJimPLaxCiOos/64jnjfeADHzj18573vOf2TOC2t73tdC+/IuLH7J1U/zxEvglwzHyRM6ubPq+yIXVgtEPzYEgOnzKkBQjucY97bPsApAkYDJDUZv+V4wxmbnwetQqC6lfBo/o7n0OPzFO3PgcWmqYy2tMHF0YBjvuc0QJRzR5GDaI/GFkoGRiMTX9f8YpXbD35yU+e+gpYQN2Yen6Mcp6MtLRtoViy6XMHY47QpGu+UmawSax7Bnn7299+GjANkA+QY5gzpezYBoJyrMbFmXFtvhgAxjQtAwbM8LmoI6Cs8lnSNKPP0vx9Tij3R6nTJjADjDHqa/GCu93tbhPgW1OgGfTLPVFKNND2mMSyyWXjVdphpdM4b2w39VJFo3YABAN6+tOfvj1w363GGfQYbUMI6rDnMbE65zH0xzzmMRPj/c5X0A7GKtOcP1Mw9glhI+5SBHApBwGj2Pye00Ztap80+475OYnAo35AmDuitA2Ae1756AwwxokOhxUDWKf8YMpPXFPf65XkkQAEbHGGmkVUTJkPFpHnTtq45AlESbw6DAiolrJ6mqqlrsd5vzZGVcs718+kHvjcGyUwDdA9s5U0lrEV3XvKU54yATxzxv5Hg2YttAFnNYYAiIim9pnMzMLoV2njqBJGTx/r3lQHdRjzEN4VQUkHQpCK/pOUEi4Q1X/TpYI7nh3n1rxlxMZsIJirx1FDICjijfF+YMMEwAGEEWirporjMnP90E9SnJ+gDDAELmPVljFgNMD7DVMBXP8TiDGoRGuhjd9G7XUy8gVP5/n+slZHRzEHUHzcgBGqaRLJzoGLGdlfhBoDNXMpBIw8/xiqvjHEq/2exwRMG2cZ1YMhmRtaA0P1ZwR+YzcGoMJ46t3z7qlX31/+8pdvS+8jHvGICeDlFKgbIPQx7REg/d69BGuMZwDBEeQAro4D5FjNM2lHdLo3agXlEfOxj33s9rMcvwaJcMBw97vf/QT1nKSl6nnRASov3WVaVXsY4jmMC3Q0UkRzX3uj3cdgzDSO4goANuYMag8YMW0EdBczoU7+TM6r+vRZeJqEjyZUPwoAod2YlFKbnil6ONfCO2UAHcJeivXCiWXYjIxtTjuGd6HdYBHG4O9yl7tM1zyxUhmSUraP//e9732nOtIAowRhCsICUH0ad/zm0SN8UjUyMjWvr54rlpBfURJqY64OjCpBRH+14XdMor1cTTlphQJVL3vZy7bNDl+BFhyDPSMo9mLzD8E/2N+DBtZ056lPfeokIQb/qEc9aiJU6rSYu6lS6/l+wwjBkqZsmOh39pFWUL68AM+bdo1JnhiEiWkLqrmEkREYo/1Vr37nC2CAumis7pFk5kVbxqZPlo+rR/3G7Xf1ua8d40IDUq3PfJlWK2kK4PDs8573vG3QaYdTqI1M3VEmi5zgBK5qeAlxSROCx

### Phasor Analysis

In [16]:
// WIP
param.paramMap = null
phasorRslt = op.run("slim.fitPhasor", param)

net.imagej.slim.FitResults@19498cc4

## Other settings

### Region of Interest

Sometimes, instead of the whole dataset, only part of the image (e.g. the region near the nucleus) are of our interest. By specifying the `roi` parameter, we neglect unwanted parts outside of it during fitting. This greatly improves the running time on large images.

In [17]:
import net.imglib2.roi.geom.real.OpenWritableBox

min = [ 20, 20 ]
max = [ 100, 100 ]

// define our region of interest, in this case [40, 87] * [40, 87]
roi = new OpenWritableBox([ min[0] - 1, min[1] - 1 ] as double[], [ max[0] + 1, max[1] + 1 ] as double[])

net.imglib2.roi.geom.real.OpenWritableBox@1d29

We start the fitting routine the same way as before but with the `roi` parameter:

In [18]:
// fitMLA with roi
param.paramMap = rldRslt.paramMap
mlaRslt = op.run("slim.fitMLA", param, roi)
nb.display(fcd.tableDisp(mlaRslt, 0.13, 0.25))

Z min = -0.23343004286289215
Z max = 5.321786880493164
A1 min = 0.0
A1 max = 1088.684814453125
Tau1 min = 0.0
Tau1 max = 3.1886990070343018
brightness_max automatically set to 921.0847


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAlr0lEQVR42u3dZ69tZbnGcUCKFCkWVEBQmmABQYp0EBUsiIogIl2KFCmChaKiYkEEFFFUpL+QIggE0BASXsBn4AsQX/Aphvk9Of+dGc/WczxZnLXnZqxkZMw1ylPu67rLU8cGG2ywwTQfr+tjFsJMgPmYCTAfMwHmYybAfMwEmI+ZAPMxE2A+ZgLMx0yA+ZgJMB8zAeZjJsB8zASYj5kA8zETYD5mAszHTID5mAkwHzMB/s/HpptuOm222Wbj91vf+tZpl112mbbbbrvpbW9727i31VZbTdtuu+30jne8Y3rXu941nnF9yy23HM/tt99+09Zbbz1ttNFG451NNtlkeuMb3zgdeuih07vf/e5pr732mj70oQ9NH/vYx8azn/zkJ6cDDjhgOu6448b5ne9857TNNttMO+200zj22Wefkdcee+wx7bnnntP2228/veUtbxlp+r3bbrtN73nPe6Y3v/nN4315y9fzrr/pTW8a5VOO1z0BNtxww/927Q1veMOa30AjSNeAC2iCdg1wiAFwQLsPGKD67RrwEOTtb3/7AGKLLbYYBxB23XXXad999x3n448/fjryyCNH+h/96EcHoO9973unHXbYYTr22GMHeJ7bf//9R76A96xn5Ck9xHTPee+99x7lVw5AS2fnnXce7yCB8qjbxhtvvFa5rMPkWNkEVXQR8LWRYccddxyAOwOUNjkTLqBonQPwnnePsAl9UUtpb4SLGKVJaz3rvUBFBL9pv7QPOeSQcd87kQ0h5AN0hzSUVV6037sshnQOOuigQSIk2H333UfZkVJ5lGV2Af910GoaldkHGsEDkoDf//73Dw36wAc+MIToeN/73reGKIRPAz1D0AgDYGAw9Ugife+5l1XwHmviGsA9A2RayjUAmtYCTRkOPPDA8RsZgO++PA8++OCRNusSAf3vnvc+85nPjPJ5T9qub7755mvq+785WMH1kgAJgUYAw/8AoUV8+oc//OFhZmkRwAke6Myq/wHnIGBaR/O5ApqHJN4HClJ5xzWgAEM68gU8UFgGz8iDzyZ0aSGSZ5Dr8MMPH+8iD7IgBbLIH/CRTzpcjPzURRmk6R0kUtdAXZv5R7z12gIA15lGOGhk2ghIAkQGQvrIRz4ytMv/wAAQgAncAVDPAxsppE07ab/AT5DnGkE7CvLkhzyRDziIRns9D2T5+A08Fkg+AJY+csgzny8teUpr0S0hMNCRSPre84z7nque/xPoq2QFXpuEFytLuEwizSJAgNIc/zOvQCZE2keb+GHaSKvSfv6WD/aOa4QLPJG+/2k44Qc20JDA/4B2BjwCffCDHxzpKIf/5Y2ETLn8ve+sXPIu9siKeN7/SO0ZdWSZ5C9f15Xfe8qg7kjxuosBAK/itJuGZG4JlZYTdH61gK/IHEgFW0yw697xG5Du09YjjjhigEnLCL8gTFryP/HEEwcYrnEJDuSTH41HQMBJwyGfT3ziE9Nhhx22xsV4Xzk8L38WQllqNUgTORzKhCSsDXKxCsU//y44Xq8IAAQ+GeCEQgOBQaCER0COAiZm3HWAE3ZaCQyCRJSjjz56jTZKu+aXSJ5lYBEWTT2hKwftc43wmXrpO6TNHdFcpOTrvaevwLvK5P0OboYVko93ihmA7776uS5NZfOcfFkIZHMPCZTnn+W1tmtLTQDAApSmqjSwaQ5tAjLgaB6NAiAr4H7anPa4jgiusQpAAnY+nIDl8/GPf3wAwSoUvTukUWSfby6Aq6zyQkwkAChCRihpKS9iucZF5bqAxmc7Azli1xSVrrSQVnn9Lz/E+Wd5Kdt6QwAVrJestjSQnQkR6HwvwZx66qlDcA5WIu0BPI0Bqg6dL37xiwMQYEqXMIGJBPwtwhVw+d97AAUQktQSAHy9jIsuqRYJTc3VABqwyuwsRiio9J40Cv48L/0sHOuhPDqcajUomzyU9d+1DpaeAIBkKglaMwmYAj/aQ2A0n8BPOeWUcc3zwCaMfDBB0SzPCcwI0vPIwxIAmDUhSMInXFrHDGeSaV3dy8w/i1DLwzOeBaa8AOcMVOVhCZCMf5cuEiBenU/S8k79DnVqeY4Vi6yIr041GZFTHbzvXYGycqxXBCA0AswfAodGpP1F+wTD7CMIIRCw37QlF4A4rhGgM6E76rGjkQhFi8UICJE5zdRK228guYcMtQ6QTj60spaC8tVppOyu13cQgdxXhk9/+tPjWm7KdeVAAlYLSdXH/4itrJ6XpnzXSwuAAMCqD9+ZJjP5tIIgIwnfnTmlFQAQfdczSMCA9v+FF144CJHpli4tAl6tB1oMXKDTttrviChvZWBZXKtzCqG8C0T3cw+ABKpyuF/3sXyVXdlKW31ZPVqO7N/61rdGS0ITtSajetWdXHywtnGTpScAgdZHT+hAc42ACIfAVL4OFGawNjIAgQIAgmIh6r+vP5/Wp50FWcUE8gWU93MJ7tWnX0SPGHXoNGYgrbqMnR0NOkmzZqTr8uGWAMzcI4DyHXXUUaOe0nTNAXhE8lzdyCwDayg9cvpXA0hLSQCROMCdCYj5+9znPjcEEaiEsdhmdtA8WsuM+x/In//858d7iJTJryOGFvZ+Zh+wfhNykbpnpFnABnjXEcO7SCItJEG28q456b6yul8TFXjq6Do/D3RgMvF+ew7Iys16IERNT/l7n3woxCqDv/IEADoQmG0VPOaYY6YvfOELQ0NpSQMttKiBHlpFQH4jASHRMKDRIlpeS4KV8B4iAQkI4g2/HX4DseZVnTy0WCTvnrP8XJc2MJTB+9Ln5xEHODU7G/lreLjgFJjSAD7rAWzPICzXRw5iHkSWxkknnTSUQrk8TyaLXcDKsPQWAJCAQwAHgX/pS18aQuVbi+T9X48fgGmos9jgtNNOG2DRNAFVYwTebdJIFoBW+p9bQQykQLjmDwAkjc5NSLeh4tJhQQDY8DLApZ1bq0XjkI+yszbALbhDePmpB/DrwpYXqwB81z2jfDUrV9ESrGyCgAagin32s58dAkAAGqbSCQIxBErf/va3x30BVNrofUIjaIATNoESdkPC0qeZjcc3+MJtyKc5BZ7x7mKzsFFB4CKANGkhEgkopVd/hXOdR/JHVnk4qwNA6+3z3gknnDDK639kcHhOPdRbmdSrYFJ95VtA+J8MIa+TBCC0mj+A5f9prt+0flGrmNBLLrlkPI8ANbka/CHMM888c9z3fAElTQccsBzcRk3F3II86uoFBu3O5WRNGlfw27OE33QvzwLJWb7qIfJvbEAZGiKWFwC5D/+754zIwEdA8QVSN9jUdDRl9Z56rFKzcGUTZOIJSi+fil988cXTRRddNH7XfCIEh/51gRLhMKPiB+eCOL5zsROG5tJIQDkAUG9ewOfL64coevccTUNQv1mLWg6NV9S7CFCHPJCg3rxmGAFYoKicjVz6HdDqoeOKJVJe5KEAWgnKqh41iRvDaF7jKowLrGyCZ5xxxnTdddetiXi/9rWvjcoSCHIQBHA+9alPTV/+8peHhrhOwITIJNJ+whI8pi0FVU0bI8gGbLzrOQABhkUAnrN7NemcWYqaajXp/C89IJSXMnofIb7yla8MQtTMZQXUjbsrT+bfdcEgy8ZN+F99f/zjHw93p84sWr2DniUb+SCVOqjb0ncEnX322UNop59++ugUocUFf6JgwnMGMMHQHPccXEEjfIHEj7IohJX5BXymvoANaICmtVmF5ho2H0B6ytIYgmfcRy5p0EhBZum6h4zIqkwsSBahuQvScabtzD7LJi+kV89zzz13Ov/889d0gLFc8pVWw8VNTll6F6DymkiABC4SqDBN+e53vztcgPsEQhi02LPAJQT+mm9uSJdVcF+6hAwQQDcrqJk7zCehN1GEgHMRzRdkFRBhccwB0Ew

In the results above, all other regions outside the box is neglected.

### Binning

Binning settings are enabled by setting the binning kernel parameter. The kernel can be any image. Here we use the built-in `SQUARE_KERNEL_3`, a $3\times3$ image with each pixel valued $\frac{1}{9}$:

In [19]:
import net.imagej.slim.SlimOps
SlimOps.SQUARE_KERNEL_3

In [20]:
import net.imagej.slim.SlimOps

// spin!
rldRslt = ij.op().run("slim.fitRLD", param, roi, SlimOps.SQUARE_KERNEL_3)

param.paramMap = rldRslt.paramMap
mlaRslt = ij.op().run("slim.fitMLA", param, roi, SlimOps.SQUARE_KERNEL_3)

nb.display(fcd.tableDisp(mlaRslt, 0.13, 0.25))

Z min = -0.054822858422994614
Z max = 3.1212329864501953
A1 min = 0.0
A1 max = 921.3705444335938
Tau1 min = 0.0
Tau1 max = 1.1913063526153564
brightness_max automatically set to 817.89197


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAmZUlEQVR42u3dZ8w/RbnGcSwgKiKiggVBUFRUilhQVCwgFuzYUBRsiBURe8GCBRv2gggWsCB2sTfsXRBjByMgCQGBl/puTz6TfMnmfzTmvNz/2SeZ7G/32TI713Vf9z1lZzbZZJNNpjX9v05rIawEWNNKgDWtBFjTSoA1rQRY00qANa0EWNNKgDWtBFjTSoA1rQRY00qANa0EWNNKgDWtBFjTSoA1rQRY00qANa0E+D+lK13pSiNd7WpXm65+9atP17rWtabrXOc6001ucpNphx12mHbeeedp++23n25605tO22233bTTTjtNt7jFLabb3OY2061udatpxx13HP+/+c1vPo7bv+ENbzhd//rXn6573euOY3vuued0j3vcY7rXve41PfzhD58e9KAHTY985COn+973vtPDHvawcXy//fab7nrXu053utOdRrr73e8+7bPPPuPaXXfddTzPveTDMzz7tre97XjuLW95y3HMVnJ8l112GfmSh2tc4xrTZpttNm266ab/6/2vcpWrTFe96lWnK1/5yv+1nDZaBVAwW2655XS9611vgKzgAC8peIUsIUXHFfKtb33rUdCA2W233abdd999HLvxjW88Cl4CzO1vf/vpbne727T//vsPAjzkIQ+ZHvGIR4z9Rz3qUdP973//QQrnAN723ve+99giw13ucpexdZ+A96w73vGO0x577DG2no8sfjvfVt6ReNtttx0kQHLvGpiRH/gSMvwnoBFo0QToJTdk/+abbz4KBwFYrQJWuHe4wx2G5Sl0W4Ue0EBWuArfec5x/Ha3u90VhLjRjW40lMDWMffYa6+9BsB77733sHbns/JAk+w7B+CO3/nOdx7n+u18eQvozum3fHiGa+55z3uO5PjNbnazQW7qtsUWWwwibEiAVFB59P/OSSUWTYBedE6IgN9qq62GtbIWQAWQwgQwcBFAAjoQEAAQQHOuwncukBR68kw1yLJ7uNbWtSX3cL7reo7jybn75g48q/vLI4UAMsAphu2+++470gEHHDBcjLwh4A1ucINBcO4tl5BRZPWAlublNFeIjcYFeElSeO1rX3vaZpttRuGQbQWugBWkwlXoChowCjGLT5IBo+CdhxQpBmkn6wrfPRHL9RKF4UK4htwJRbGlHEjDYos7IqQYAiHlYZ4/W6n/S57r+RHAfbm2kndGhNRgQxJsaCjKyvGNggCxnKQpADKt0BWsgla4WRcgAawQk+KSwr/Pfe4zfLfzFDyi2Dr26Ec/egDgWCoAiAI5x5DAs3M7rB/giOI6vz3rfve73wDUMxEM2MgneCxGkD/b1Ms+del5SI6IgljJPtWjgNe85jWvCATnMUIK4H+LdwG9jBfE+q233noUQlF9Fq0AgYgAInOFHSlIK1CRAuiAiQD+Dxz+WJBXlA9c/pfVkWDPZN2RwfMBVGAXgLblCdEe8IAHjPvJk+c6Tv6Bjgh+OwZ0iRuhLEjWMzyv+CVSKAexAZcwd5P/jgSLJ4BqHsaTfiBIAFIgWb3CI/NAUPASwG1ZYarAGoHB1wIHWLkBQDnuXAXuOVxM1cNUBziUQOJ6kvos2DM866EPfeh4tm15QjTARw7XyQMlk4DNlXiurVTcIY/2PVcNoQBR+cylP7XcMH5abC2A1HlZhZLvLYIu0CoiZ1H50axLIQNdqu7unAc+8IEDPAQg9X6zUuC4NzAcBzoZrq5ebQFBXFswWEAoL8iImNTnwQ9+8MiP39QGOXI3nucY9+HdEFvyjH5XdS2mkQ9kpAKi/zkBWDxVUGbVFhZNAC/nZTFfYVRtAy6gFGDgKuQkF8D53AIs4GeRBx544AAGeRQugN0fmFlyBZ/VA0g+nIuMjteOUIBYzQEJcgvyVuQP7FRJY5LjBZOuj2A1IiGSJD9VFamdfHAF4iFlhAhzAnCXGwaGiyQASwOMQgBOEs5fAxGYj33sY6cnPvGJ05Oe9KSxny+urq1QkYa0P+EJT5gOP/zwcS4LdEyh1iKo0LPmrBIwteLxy36nCMADRgFi7Q3k3PVAo0y1KFKBOWmTf+e6T/d0TfEM4sin693H/7yT/CCBpEo8bzmcB82LJoCCyZq8OB8NZC1yLEgCJEAPPvjgKyJ4FgRExIkA/nfooYdOT3/60wcRas6lEp2DCCws+eVrFTAXVAReZA4whEBS1bR5i6NrAcnigRwRckHVDPLv/l87gfzbUgpBqfc65JBDBuG5LS7EfdzPeyIuRVJFpATVDIoHFk2A/Hh+nvUUrSMCGVcwCsm2OrxCqdALDp0L+Gc+85nTU57ylHFNcl9bAPCcnw8GPBcEeGRQK0ASoCEnNYgcSJCbcj/J7xqbpIC2rfooj0gxr7qyeO9IrZ73vOdNRx555PT4xz9+vDMScHVSzc4IKR/cQQqwUcQAJLMGHJZZHV5bfBE7a7FVGKy4KlTNrwrUdVSDWjzucY8bZFGA5DcwnVujUgQAOAsXcNX4VHMyC08VkCTfXVu/+9UKWYvkPMCjII7Jt74G4MoT60ZWeUUA4D/nOc8Zbs55xTkFloxDnuSTG9iwgWjRBACUF/SiQD7ooIOGzyeJJLyInUrU/EuKFUZ+mgUqNMCzfO4CGVgdwACZxbJu/tz1ZFWkXwuf+9m6H5IAukYax11fwFZdP/Crx3f/Whid772QWgpY1u89n/a0p01PfepTR7LvnFyI6wqEkQipkHGj6g1kFaTRSyuUZz3rWdOTn/zkId9eXFAEjLkl14DCKv1WyArONdJjHvOYUXiI43xKUWNQIJFzJHB9QR6Q7QO66l+uo16+uS+Xr+4XGf1OFeo08h4ICWCEp2wUgKubJ//zHsjsGgZACalC1UnqV42gBrTFK0Ddrqz+uc997vTsZz97BHNefN7UWocLy8sPA5/PTDmoQO4jkGqFC7AietuCtEgFxAgQ2J5XT988OacqJPK4j3wBvbYJRAS2d0HwQK4ZWT6r7QDcuf6PELkz11EC96U0YgHAS4uvBQBOAXh5QZAATiCnwCrA2gAqRNdwGZRDIZJ8ieW7FwIowGKGyFJ1rEg+12JrHwFSm7n1Vz93ri3SOZ+KCByRhzzXA1negAxY+UVo7wdUAEcQ53inajsUDJFdIyEAgjOCOp3kU0PQho1EiyRAoHl5wPPhlKBIuIJ0jkDpqKOOGlGzglWQLIbL4Eulww47bBRargUgAGM51ekFaWS6Jl3nFPRl2TXLAjoXUtuD/+WCIgDr9798N0Dz8/Ke5Qd+vYdiCXmVgO9dlEPXIXJVwlwYtdFErDFIULhoAgBcJKzufsQRR0wvetGLhgvI77ESILOeo48+ejrmmGOml7/85YMkc/UAOhVQgEhV55BCZtURAFiNHKoHsZE7AK9lsOFczs0lIFJEQQDg14pZuwSiAAyB5QuAFIGE1x2NJM5FBOcVu6gFILCAEJmVgetYPvJJnsN9aSFs0MyiCcCan//85w/rtn3pS186veY1r7mi4ScZJaH+94pXvGKcy0Iax+f/Ck/BOU+hKnTkYTl16TYmDwlq4m14FhAD3D6AHasnsJE9jUTKBUSa2gzECpEWCWuIAr592+IR/xMcyjfggU7hGAGyM4p6L93DO9VMrDag6qpxaNEEAOgb3vCGkY477rjpLW95y9iqDVACJFBAQGf5L3vZy8Y+RSjw4z5YPWtT8AIrvxGBAtRql5XWHNvgUbWBWvsKBiVKQB3EEoB3HcI05KvOm8YKcC2e57meHwFr9XPctmZgscoznvGM0Q4g+AU4hVMGb3rTm0YZULiaixGgRixuQH4FhIsmgBd+3eteN73+9a8fL/2qV71qsJ87ADaFYBEvfvGLp1e+8pWjoFg7y1GgNRtXUBRDweYKFHTASXXs1BTcyCDAAzwX4N6kluTnf53